In [1]:
pwd

'/Users/dblyon/modules/cpr/agotool'

In [2]:
cd app/python

/Users/dblyon/modules/cpr/agotool/app/python


In [3]:
import os, sys, socket

In [4]:
sys.version, sys.executable

('3.8.6 | packaged by conda-forge | (default, Dec 26 2020, 04:50:20) \n[Clang 11.0.0 ]',
 '/Users/dblyon/anaconda3/envs/agotool/bin/python')

# Imports

In [5]:
import query, variables, userinput

##################################################
env_file missing
##################################################


In [6]:
%load_ext Cython

In [7]:
%reload_ext Cython

# Preload

In [8]:
UniProt_IDs_human_list = sorted(query.get_proteins_of_taxid(9606, read_from_flat_files=True))
ENSP_human_list = sorted(query.get_proteins_of_human())
###
from itertools import islice

def get_random_human_ENSP(num_ENSPs=20, joined_for_web=False, contiguous=False, UniProt_ID=False, UniProt_IDs_human_list=UniProt_IDs_human_list, ENSP_human_list=ENSP_human_list):
    if UniProt_ID:
        IDs_2_sample = UniProt_IDs_human_list
    else:
        IDs_2_sample = ENSP_human_list
    max_index = len(IDs_2_sample)
    if not contiguous:
        if not joined_for_web:
            return random.sample(IDs_2_sample, num_ENSPs)
        else:
            return "%0d".join(random.sample(IDs_2_sample, num_ENSPs))
    else:
        start_pos = np.random.randint(0, max_index)
        if start_pos + num_ENSPs > max_index:
            start_pos = max_index - num_ENSPs
        stop_pos = start_pos + num_ENSPs
        if not joined_for_web:
            return list(islice(IDs_2_sample, start_pos, stop_pos))
        else:
            return "%0d".join(list(islice(IDs_2_sample, start_pos, stop_pos)))

In [9]:
import query, variables
from importlib import reload
reload(query)
reload(variables)
variables.DB_DOCKER = False
variables.DOCKER = False
# taxid_2_funcEnum_index_2_associations --> taxid_2_tuple_funcEnum_index_2_associations_counts
low_memory = True
### preload
pqo = query.PersistentQueryObject_STRING(low_memory)
static_preloaded_objects = pqo.get_static_preloaded_objects(low_memory)
### needed for scipy comparison, but otherwise deprecated
# ENSP_2_tuple_funcEnum_score_dict = query.get_proteinAN_2_tuple_funcEnum_score_dict(read_from_flat_files=True)

##################################################
env_file missing
##################################################
################################################################################
initializing PQO
getting taxid_2_proteome_count
getting taxid_2_proteins_dict
getting NCBI_taxonomy and TaxidSpecies_2_TaxidProteome_dict
getting lookup arrays
getting cond arrays
getting lineage dict
getting blacklisted terms
getting taxid_2_tuple_funcEnum_index_2_associations_counts
getting preloaded objects per analysis
finished with PQO init
go go GO and fly like the wind
################################################################################


In [10]:
year_arr, hierlevel_arr, entitytype_arr, functionalterm_arr, indices_arr, description_arr, category_arr, etype_2_minmax_funcEnum, function_enumeration_len, etype_cond_dict, etype_2_num_functions_dict, taxid_2_proteome_count, taxid_2_tuple_funcEnum_index_2_associations_counts, lineage_dict_enum, blacklisted_terms_bool_arr, cond_etypes_with_ontology, cond_etypes_rem_foreground_ids, kegg_taxid_2_acronym_dict, goslimtype_2_cond_dict = static_preloaded_objects

# Functions

In [15]:
%%cython -f --compile-args=-DCYTHON_TRACE=1

import Cython
######################################
### profiling # Set compiler directives (cf. http://docs.cython.org/src/reference/compilation.html)
import line_profiler
directive_defaults = Cython.Compiler.Options.get_directive_defaults() ### from Cython.Compiler.Options import directive_defaults # deprecated
directive_defaults['linetrace'] = True
directive_defaults['binding'] = True
######################################
from functools import reduce
import math
import numpy as np
import pandas as pd
from cython cimport boundscheck, wraparound, cdivision, nonecheck
cimport cython
cimport numpy as np
ctypedef np.uint8_t uint8
from collections import defaultdict
from fisher import pvalue
from scipy import stats
import variables, query
import time
import colnames as cn



def run_enrichment_cy(ncbi, ui, preloaded_objects_per_analysis, static_preloaded_objects, low_memory=False, debug=False):
    if not low_memory:
        ENSP_2_functionEnumArray_dict, year_arr, hierlevel_arr, entitytype_arr, functionalterm_arr, indices_arr, description_arr, category_arr, etype_2_minmax_funcEnum, function_enumeration_len, etype_cond_dict, etype_2_num_functions_dict, taxid_2_proteome_count, taxid_2_tuple_funcEnum_index_2_associations_counts, lineage_dict_enum, blacklisted_terms_bool_arr, cond_etypes_with_ontology, cond_etypes_rem_foreground_ids, kegg_taxid_2_acronym_dict, goslimtype_2_cond_dict = static_preloaded_objects
    else:  # missing: ENSP_2_functionEnumArray_dict
        year_arr, hierlevel_arr, entitytype_arr, functionalterm_arr, indices_arr, description_arr, category_arr, etype_2_minmax_funcEnum, function_enumeration_len, etype_cond_dict, etype_2_num_functions_dict, taxid_2_proteome_count, taxid_2_tuple_funcEnum_index_2_associations_counts, lineage_dict_enum, blacklisted_terms_bool_arr, cond_etypes_with_ontology, cond_etypes_rem_foreground_ids, kegg_taxid_2_acronym_dict, goslimtype_2_cond_dict = static_preloaded_objects
    foreground_ids_arr_of_string, background_ids_arr_of_string, funcEnum_count_foreground, funcEnum_count_background, p_values, p_values_corrected, cond_multitest, blacklisted_terms_bool_arr_temp, cond_terms_reduced_with_ontology, cond_filter, cond_PMIDs, effectSizes, over_under_int_arr, over_under_arr_of_string = preloaded_objects_per_analysis
    em = ui.enrichment_method
    foreground_n = ui.get_foreground_n()
    args_dict = ui.args_dict
    background_n = ui.get_background_n()
    protein_ans_fg = ui.get_foreground_an_set()
    taxid = args_dict["taxid"]
    filter_foreground_count_one = args_dict["filter_foreground_count_one"]
    p_value_cutoff = args_dict["p_value_cutoff"]

    if ui.enrichment_method in {"abundance_correction", "compare_samples"}: # , "compare_groups"
        protein_ans_bg = ui.get_background_an_set()
    if low_memory:
        ENSP_2_functionEnumArray_dict = query.get_functionEnumArray_from_proteins(ui.get_all_individual_AN(), dict_2_array=True)
    ### add protein groups to ENSP_2_functionEnumArray_dict
    ENSP_2_functionEnumArray_dict = add_protein_groups_to_ENSP_2_functionEnumArray_dict(ENSP_2_functionEnumArray_dict, ui.get_all_unique_proteinGroups())

    count_all_terms(ENSP_2_functionEnumArray_dict, protein_ans_fg, funcEnum_count_foreground)

    ### count background
    if em == "genome":
        funcEnum_index_2_associations = taxid_2_tuple_funcEnum_index_2_associations_counts[taxid]
        funcEnum_index_positions_arr, counts_arr = funcEnum_index_2_associations
        create_funcEnum_count_background_v3(funcEnum_count_background, funcEnum_index_positions_arr, counts_arr)
    elif em == "abundance_correction":
        funcEnum_count_background = count_all_term_abundance_corrected(ui, ENSP_2_functionEnumArray_dict, funcEnum_count_background)
        background_n = foreground_n
    elif em == "compare_samples":
        count_all_terms(ENSP_2_functionEnumArray_dict, protein_ans_bg, funcEnum_count_background)
    else:
        args_dict["ERROR enrichment_method"] = "The 'enrichment_method' you've provided: '{}' doesn't exist / isn't implemented.".format(args_dict["enrichment_method"])
        return args_dict

    ## limit to given entity types
    cond_limit_2_entity_type = limit_to_entity_types(args_dict["limit_2_entity_type"], function_enumeration_len, etype_cond_dict, funcEnum_count_foreground)
    limit_to_go_subset(etype_cond_dict, args_dict["go_slim_subset"], goslimtype_2_cond_dict, funcEnum_count_foreground)
    o_or_u_or_both_encoding = args_dict["o_or_u_or_both_encoding"]

    ### calculate Fisher p-values and get bool array for multiple testing

    calc_pvalues(funcEnum_count_foreground, funcEnum_count_background, foreground_n, background_n, p_values, cond_multitest, effectSizes, over_under_int_arr, o_or_u_or_both_encoding)

    ### "over/under"
    if o_or_u_or_both_encoding == 1: # overrepresented
        over_under_arr_of_string[over_under_int_arr == 1] = "o"
    elif o_or_u_or_both_encoding == 0: # both
        over_under_arr_of_string[over_under_int_arr == 1] = "o"
        over_under_arr_of_string[over_under_int_arr == 2] = "u"
    elif o_or_u_or_both_encoding == 2: # underrepresented
        over_under_arr_of_string[over_under_int_arr == 2] = "u"
    else: # check already done above
        return args_dict
    ### multiple testing per entity type, save results preformed p_values_corrected
    if args_dict["multiple_testing_per_etype"]:
        for etype_name, cond_etype in etype_cond_dict.items():
            if args_dict["multiple_testing_stringency"] == "A":
                num_total_tests = p_values[cond_etype & cond_multitest].shape[0] # sum(cond_etype & cond_multitest) is prohibitively slow!
            else:
                num_total_tests = etype_2_num_functions_dict[etype_name]
            multiple_testing_per_entity_type(cond_etype, cond_multitest, p_values, p_values_corrected, indices_arr, num_total_tests)
    else:
        cond_all = np.ones(function_enumeration_len, dtype=bool)
        if args_dict["multiple_testing_stringency"] == "A":
            num_total_tests = sum(cond_multitest)
        else:
            num_total_tests = cond_all.shape[0]
        multiple_testing_per_entity_type(cond_all, cond_multitest, p_values, p_values_corrected, indices_arr, num_total_tests)


    ### Filter stuff
    foreground_ids_arr_of_string, funcEnum_indices_for_IDs, cond_etypes_with_ontology_filtered, cond_etypes_rem_foreground_ids_filtered, cond_filter = filter_stuff(args_dict, protein_ans_fg, p_values_corrected, foreground_ids_arr_of_string, funcEnum_count_foreground, year_arr, p_values, indices_arr, ENSP_2_functionEnumArray_dict, cond_filter, etype_cond_dict, cond_PMIDs, cond_etypes_with_ontology, cond_etypes_rem_foreground_ids, over_under_int_arr)
    if debug:
        return foreground_ids_arr_of_string
    if em in {"compare_samples"}:
        background_ids_arr_of_string = map_funcEnum_2_ENSPs(protein_ans_bg, ENSP_2_functionEnumArray_dict, funcEnum_indices_for_IDs, background_ids_arr_of_string)

    ### filter etypes with ontologies --> cond_terms_reduced_with_ontology
    df_with_ontology = pd.DataFrame({"term_enum": indices_arr[cond_etypes_with_ontology_filtered].view(), "foreground_ids": foreground_ids_arr_of_string[cond_etypes_with_ontology_filtered].view(), "hierarchical_level": hierlevel_arr[cond_etypes_with_ontology_filtered].view(), "p_value": p_values[cond_etypes_with_ontology_filtered].view(), "foreground_count": funcEnum_count_foreground[cond_etypes_with_ontology_filtered].view(), "etype": entitytype_arr[cond_etypes_with_ontology_filtered].view()})
    if args_dict["filter_parents"]: # only for etypes with ontology, but since foreground IDs needed get them for all
        filter_parents_if_same_foreground(blacklisted_terms_bool_arr_temp, cond_terms_reduced_with_ontology, lineage_dict_enum, df_with_ontology) # modifies cond_terms_reduced_with_ontology inplace
    else: # since no filtering done use all etypes with ontology
        cond_terms_reduced_with_ontology = cond_filter & cond_etypes_with_ontology
    ### concatenate filtered results
    cond_2_return = cond_PMIDs | cond_terms_reduced_with_ontology | cond_etypes_rem_foreground_ids_filtered

    df_2_return = pd.DataFrame({cn.term: functionalterm_arr[cond_2_return].view(),
                            cn.hierarchical_level: hierlevel_arr[cond_2_return].view(),
                            cn.p_value: p_values[cond_2_return].view(),
                            cn.FDR: p_values_corrected[cond_2_return].view(),
                            cn.category: category_arr[cond_2_return].view(),
                            cn.etype: entitytype_arr[cond_2_return].view(),
                            cn.description: description_arr[cond_2_return].view(),
                            cn.year: year_arr[cond_2_return].view(),
                            cn.FG_IDs: foreground_ids_arr_of_string[cond_2_return].view(),
                            cn.FG_count: funcEnum_count_foreground[cond_2_return].view(),
                            cn.BG_count: funcEnum_count_background[cond_2_return].view(),
                            cn.effect_size: effectSizes[cond_2_return].view(),
                            cn.over_under: over_under_arr_of_string[cond_2_return].view(),
                            cn.funcEnum: indices_arr[cond_2_return].view()})

    # cols_2_return_sort_order = cn.cols_2_return_run_enrichment_cy[:]
    cols_2_return_sort_order = list(cn.cols_2_return_run_enrichment_cy)
    if em in {"compare_samples"}:
        df_2_return[cn.BG_IDs] = background_ids_arr_of_string[cond_2_return].view()
    else:
        cols_2_return_sort_order.remove(cn.BG_IDs)
    df_2_return["s_value"] = get_s_value(df_2_return)
    # df_2_return["s_value_abs"] = df_2_return["s_value"].apply(lambda x: abs(x))
    df_2_return["s_value_abs"] = np.abs(df_2_return["s_value"])
    df_2_return = df_2_return.sort_values([cn.etype, "s_value_abs", cn.hierarchical_level, cn.year], ascending=[False, False, False, False]).reset_index(drop=True)
    df_2_return[cn.rank] = df_2_return.groupby(cn.etype)["s_value_abs"].rank(ascending=False, method="first").fillna(value=df_2_return.shape[0]).astype(int)
    if debug:
            return protein_ans_bg, ENSP_2_functionEnumArray_dict, funcEnum_indices_for_IDs, background_ids_arr_of_string, df_2_return
    df_2_return = ui.translate_primary_back_to_secondary(df_2_return)
    df_2_return[cn.FG_n] = foreground_n
    df_2_return[cn.BG_n] = background_n

    ### calc ratio in foreground, count foreground / len(protein_ans)
    df_2_return[cn.ratio_in_FG] = df_2_return[cn.FG_count] / df_2_return[cn.FG_n]
    df_2_return[cn.ratio_in_BG] = df_2_return[cn.BG_count] / df_2_return[cn.BG_n]
    if args_dict["STRING_beta"]:
        return df_2_return.rename(columns=cn.colnames_2_rename_dict_STRING_beta)[list(cn.cols_sort_order_STRING_beta)]
    elif args_dict["STRING_API"]:
        df_2_return["ncbiTaxonId"] = taxid
        df_2_return["preferredNames"] = ""
        return df_2_return.rename(columns=cn.colnames_2_rename_dict_STRING_API)[list(cn.cols_sort_order_STRING_API_genome_or_compare_samples)]
    return df_2_return[cols_2_return_sort_order]

@boundscheck(False)
@wraparound(False)
cdef calc_pvalues(unsigned int[::1] funcEnum_count_foreground,
                  unsigned int[::1] funcEnum_count_background,
                  unsigned int foreground_n,
                  unsigned int background_n,
                  double[::1] p_values,
                  cond_multitest,
                  double[::1] effectSizes,
                  unsigned int[::1] over_under_int_arr,
                  unsigned int o_or_u_or_both_encoding):
    cdef:
        int index_, foreground_count, background_count, a, b, c, d
        int len_functions = funcEnum_count_foreground.shape[0]
        dict fisher_dict = {}
        double p_val_uncorrected
        double odds_ratio

    for index_ in range(len_functions):
        foreground_count = funcEnum_count_foreground[index_]
        if foreground_count > 0:
            cond_multitest[index_] = True
            over_under_int_arr[index_] = 3 # meaningless encoding in order not to filter out things later if p_value_cutoff == 1
            if foreground_count == 1: # leave p-value set to 1, BUT DO multiple testing
                continue
            background_count = funcEnum_count_background[index_]
            a = foreground_count # number of proteins associated with given GO-term
            b = foreground_n - foreground_count # number of proteins not associated with GO-term
            c = background_count
            d = background_n - background_count
            p_val_uncorrected = fisher_dict.get((a, b, c, d), -1)
            if p_val_uncorrected == -1:
                if o_or_u_or_both_encoding == 1: # overrepresented
                    p_val_uncorrected = pvalue(a, b, c, d).right_tail
                    over_under_int_arr[index_] = 1
                elif o_or_u_or_both_encoding == 0: # both
                    p_val_uncorrected = pvalue(a, b, c, d).two_tail
                    try:
                        is_greater = (a / (a + b)) > (c / (c + d))
                        if is_greater:
                            is_greater = 1
                        else:
                            is_greater = 2
                    except ZeroDivisionError:
                        is_greater = 0 # np.nan
                    over_under_int_arr[index_] = is_greater
                elif o_or_u_or_both_encoding == 2: # underrepresented
                    p_val_uncorrected = pvalue(a, b, c, d).left_tail
                    over_under_int_arr[index_] = 2
                else:
                    p_val_uncorrected = 1
                    over_under_int_arr[index_] = 3
                fisher_dict[(a, b, c, d)] = p_val_uncorrected
            else: # write over_under but don't calc pvalue
                if o_or_u_or_both_encoding == 1: # overrepresented
                    over_under_int_arr[index_] = 1
                elif o_or_u_or_both_encoding == 0: # both
                    try:
                        is_greater = (a / (a + b)) > (c / (c + d))
                    except ZeroDivisionError:
                        is_greater = np.nan # ??? shouldn't this be 0 instead of np.nan ???
                    over_under_int_arr[index_] = is_greater
                elif o_or_u_or_both_encoding == 2: # underrepresented
                    over_under_int_arr[index_] = 2
                else:
                    over_under_int_arr[index_] = 3 # which caser is this supposed to be?
            p_values[index_] = p_val_uncorrected
            try:
                # https://stats.stackexchange.com/questions/22508/effect-size-for-fishers-exact-test
                # odds_ratio = (a * d) / (b * c) # true odds ratio
                # odds_ratio = (d / (c + d)) - (a / (a + b)) # difference in proportions
                odds_ratio = (a / (a + b)) - (c / (c + d)) # difference in proportions DBL
                # odds_ratio = (a / (a + b)) / (c / (c + d)) # from old agotool, ratio of percent in fg to percent in bg
            except ZeroDivisionError:
                odds_ratio = np.nan
            effectSizes[index_] = odds_ratio
    return 0

@boundscheck(False)
@wraparound(False)
cdef int create_funcEnum_count_background_v3(unsigned int[::1] funcEnum_count_background,
                                         const unsigned int[::1] funcEnum_index_arr, # uint32
                                         const unsigned int[::1] count_arr): # uint32
    cdef:
        int i, N = funcEnum_index_arr.shape[0]
        unsigned int index_
        unsigned short count

    for i in range(N):
        index_ = funcEnum_index_arr[i]
        count = count_arr[i]
        funcEnum_count_background[index_] = count
    return 0

def count_all_term_abundance_corrected(ui, ENSP_2_functionEnumArray_dict, funcEnum_count):
    funcEnum_count_float = np.zeros(funcEnum_count.shape[0], dtype=np.dtype("float64"))
    for proteinGroup_list, correction_factor in ui.iter_bins():
        for proteinGroup in proteinGroup_list:
            try:
                funcEnum_associations = ENSP_2_functionEnumArray_dict[proteinGroup]
            except KeyError: # no functional annotation for proteins
                continue
            count_terms_cy_abundance_corrected(correction_factor, funcEnum_associations, funcEnum_count_float)
    funcEnum_count = np.around(funcEnum_count_float).astype(dtype=np.dtype("uint32"))
    return funcEnum_count

@boundscheck(False)
@wraparound(False)
cdef int count_terms_cy_abundance_corrected(double correction_factor,
                                        unsigned int[::1] funcEnum_associations,
                                        double[::1] funcEnum_count_float):
    cdef int N, i, k
    N = funcEnum_associations.shape[0]
    for i in range(N):
        k = funcEnum_associations[i]
        funcEnum_count_float[k] += correction_factor
    return 0

def count_all_terms(ENSP_2_functionEnumArray_dict, protein_ans, funcEnum_count):
    for ENSP in (ENSP for ENSP in protein_ans if ENSP in ENSP_2_functionEnumArray_dict):
        funcEnumAssociations = ENSP_2_functionEnumArray_dict[ENSP]
        count_terms_cy(funcEnumAssociations, funcEnum_count)

@boundscheck(False)
@wraparound(False)
cdef int count_terms_cy(unsigned int[::1] funcEnum_associations,
                    unsigned int[::1] funcEnum_count):
    """
    without returning 'funcEnum_count' the function does inplace change of 'funcEnum_count'
    :param funcEnum_associations: np.array (of variable length, with functional associations 
    as enumerations (instead of strings), 
    uint32, i.e. which functional associations are given for provided user input proteins)
    :param funcEnum_count: np.array (shape of array from 0 to max enumeration of functional-terms, 
    uint32, each position codes for 
    a specific functional term, the value is a count for the given user input)
    :return: None
    """
    cdef int N, i, k
    N = funcEnum_associations.shape[0]
    for i in range(N):
        k = funcEnum_associations[i]
        funcEnum_count[k] += 1
    return 0

def collect_scores_per_term_characterize_foreground(protein_AN_list, ENSP_2_tuple_funcEnum_score_dict, funcEnums_2_include_set, score_cutoff=3):
    funcEnum_2_scores_dict = defaultdict(lambda: [])
    for protein_AN in protein_AN_list:
        funcEnum_already_counted = set()
        try:
            funcEnum_score = ENSP_2_tuple_funcEnum_score_dict[protein_AN]
        except KeyError:
            continue
        funcEnum_arr, score_arr = funcEnum_score
        len_funcEnum_arr = len(funcEnum_arr)
        for index_ in range(len_funcEnum_arr):
            funcEnum = funcEnum_arr[index_]
            if funcEnum in funcEnums_2_include_set:
                score = score_arr[index_]
                if score >= score_cutoff:
                    if funcEnum not in funcEnum_already_counted:
                        # in order to count a function only once per protein
                        funcEnum_2_scores_dict[funcEnum].append(score)
                        funcEnum_already_counted.update(set([funcEnum]))
    return funcEnum_2_scores_dict

def collect_scores_per_term(protein_AN_list, ENSP_2_tuple_funcEnum_score_dict, list_2_array=False):
    """
    ENSP_2_tuple_funcEnum_score_dict['3702.AT1G01010.1']
    (array([ 211,  252,  253], dtype=uint32),
     array([4200000, 4166357, 4195121], dtype=uint32))
    funcEnum_2_scores_dict: key: functionEnumeration, val: list of scores
    """
    funcEnum_2_scores_dict = defaultdict(lambda: [])
    for protein_AN in protein_AN_list:
        try:
            funcEnum_score = ENSP_2_tuple_funcEnum_score_dict[protein_AN]
        except KeyError:
            continue
        funcEnum_arr, score_arr = funcEnum_score
        len_funcEnum_arr = len(funcEnum_arr)
        for index_ in range(len_funcEnum_arr):
            score = score_arr[index_]
            funcEnum_2_scores_dict[funcEnum_arr[index_]].append(score)
    if list_2_array:
        return {funcEnum: np.asarray(scores, dtype=np.dtype(variables.dtype_TM_score)) for funcEnum, scores in funcEnum_2_scores_dict.items()} # float64 --> uint32
    # since concatenating np.arrays later on (for filling with zeros) produces 64 bit array anyway
    else:
        return funcEnum_2_scores_dict

def collect_scores_per_term_limit_2_inclusionTerms(protein_AN_list, ENSP_2_tuple_funcEnum_score_dict, funcEnums_2_include_set, list_2_array=False):
    """
    for a given protein: a functional term should only have a single score (not multiple as previously)
    ENSP_2_tuple_funcEnum_score_dict['3702.AT1G01010.1']
    (array([ 211,  252,  253], dtype=uint32),
     array([420000, 4166357, 4195121], dtype=uint32))
    funcEnum_2_scores_dict: key: functionEnumeration, val: list of Integer scores ( )
    """
    len_protein_AN_list = len(protein_AN_list)
    funcEnum_2_scores_dict = defaultdict(lambda: [0]*len_protein_AN_list)
    for index_protein, protein_AN in enumerate(protein_AN_list):
        try:
            funcEnum_score = ENSP_2_tuple_funcEnum_score_dict[protein_AN]
        except KeyError:
            continue
        funcEnum_arr, score_arr = funcEnum_score
        len_funcEnum_arr = len(funcEnum_arr)
        for index_ in range(len_funcEnum_arr):
            funcEnum = funcEnum_arr[index_]
            if funcEnum in funcEnums_2_include_set:
                score = score_arr[index_]
                funcEnum_2_scores_dict[funcEnum][index_protein] = score # funcEnum_2_scores_dict[funcEnum].append(score)
    if list_2_array:
        return {funcEnum: np.asarray(scores, dtype=np.dtype(variables.dtype_TM_score)) for funcEnum, scores in funcEnum_2_scores_dict.items()}
    # since concatenating np.arrays later on (for filling with zeros) produces 64 bit array anyway
    else:
        return funcEnum_2_scores_dict

def collect_scores_per_term_abundance_corrected(ui, ENSP_2_tuple_funcEnum_score_dict, funcEnums_2_include_set, list_2_array=False):
    funcEnum_2_scores_dict = defaultdict(lambda: [])
    for proteinGroup_list, correction_factor in ui.iter_bins():
        for proteinGroup in proteinGroup_list:
            try:
                funcEnum_score = ENSP_2_tuple_funcEnum_score_dict[proteinGroup]
            except KeyError:
                continue
            funcEnum_arr, score_arr = funcEnum_score
            len_funcEnum_arr = len(funcEnum_arr)
            for index_ in range(len_funcEnum_arr):
                funcEnum = funcEnum_arr[index_]
                if funcEnum in funcEnums_2_include_set:
                    score = score_arr[index_]
                    funcEnum_2_scores_dict[funcEnum].append(score*correction_factor)
    if list_2_array:
        return {funcEnum: np.asarray(scores, dtype=np.dtype(variables.dtype_TM_score)) for funcEnum, scores in funcEnum_2_scores_dict.items()}
        # since concatenating np.arrays later on (for filling with zeros) produces 64 bit array anyway
    else:
        return funcEnum_2_scores_dict

@boundscheck(False)
@wraparound(False)
@cdivision(True)
cdef BenjaminiHochberg_cy(double[::1] p_values,
                         unsigned int num_total_tests,
                         double[::1] p_values_corrected,
                         unsigned int[::1] indices_2_BH):
    """
    #!!! cpdef instead of cdef for scores debugging/profiling
    ein index array mit absoluten positionen, pvals absolut und pvalscorr absolut
    p_values_2_BH, p_values_2_BH.shape[0], p_values_corrected_2_BH, indices_of_p_values_2_BH)
    :param p_values: unsorted array of float
    :param num_total_tests: Integer (number of all possible tests within etype/category, regardless of input)
    :param p_values_corrected: array of float (1.0 by default), shape is full function_enumeration_len NOT p_values    
    :param indices_2_BH: indices of superset, shape of array reduced to p_values_2_BH
    iterate over p_values in p_values_2_BH_sort_order
    set p_value_corrected at positions from indices_2_BH[p_values_2_BH_sort_order]
    """
    cdef:
        double prev_bh_value = 0.0
        double p_value, bh_value
        unsigned int index_2_BH, i
        unsigned int enum_counter = 1
        unsigned int N = indices_2_BH.shape[0]

    for i in range(N):
        index_2_BH = indices_2_BH[i]
        p_value = p_values[index_2_BH]
        bh_value = p_value * num_total_tests / enum_counter
        # Sometimes this correction can give values greater than 1,
        # so we set those values at 1
        bh_value = min(bh_value, 1)
        # To preserve monotonicity in the values, we take the
        # maximum of the previous value or this one, so that we
        # don't yield a value less than the previous.
        bh_value = max(bh_value, prev_bh_value)
        prev_bh_value = bh_value
        p_values_corrected[index_2_BH] = bh_value
        enum_counter += 1

def map_funcEnum_2_ENSPs(protein_ans_list, ENSP_2_functionEnumArray_dict, funcEnum_indices, foreground_ids_arr_of_string):
    """
    previously named get_foreground_IDs_arr now map_funcEnum_2_ENSPs
    for given protein_ans produce concatenate strings of ENSP associations
    :param protein_ans_list: List of String (or array), user provided ENSPs
    :param ENSP_2_functionEnumArray_dict: key: String, val: array of uint32, all ENSP to function enum associations
    :param funcEnum_indices: array of uint32, relevant func enums after filtering
    :param foreground_ids_arr_of_string: list of empty string, len of function_enumeration_len, list instead of array since len of longest string unknown and would take lots of memory
    :return: List of String of len function_enumeration_len with comma sep ENSPs at index positions coding for func enum
    """
    funcEnum_2_ENSPs_dict = {index_: [] for index_ in funcEnum_indices}
    for ENSP in protein_ans_list:
        try:
            functionEnumArray = ENSP_2_functionEnumArray_dict[ENSP]
        except KeyError: # happens since some ENSPs are without functional associations (or if single association in genome it is filtered out)
            continue
        for funcEnum in functionEnumArray:
            if funcEnum in funcEnum_2_ENSPs_dict:
                funcEnum_2_ENSPs_dict[funcEnum].append(ENSP)

    for funcEnum, ENSPs in funcEnum_2_ENSPs_dict.items():
        foreground_ids_arr_of_string[funcEnum] = ";".join(sorted(set(ENSPs))) # needs to be sorted otherwise grouping incorrect later on
    return foreground_ids_arr_of_string

def get_preloaded_objects_for_single_analysis(blacklisted_terms_bool_arr, function_enumeration_len=6834675):
    """
    funcEnum_count_foreground, funcEnum_count_background, p_values, p_values_corrected, cond_multitest, blacklisted_terms_bool_arr_temp, cond_terms_reduced_with_ontology, foreground_ids_arr_of_string, cond_filter, cond_PMIDs
    """
    funcEnum_count_foreground = np.zeros(shape=function_enumeration_len, dtype=np.dtype("uint32"))
    foreground_ids_arr_of_string = np.empty(shape=(function_enumeration_len,), dtype=object)
    blacklisted_terms_bool_arr_temp = blacklisted_terms_bool_arr.copy()
    # was uint32, but uint16 is sufficient for STRING v11, not using it for the foreground due to potential redundancy
    # or for "compare_samples" for the same reason --> keep the same
    funcEnum_count_background = np.zeros(shape=function_enumeration_len, dtype=np.dtype("uint32"))
    p_values = np.ones(shape=function_enumeration_len, dtype=np.dtype("float64"))
    p_values_corrected = np.ones(shape=function_enumeration_len, dtype=np.dtype("float64"))
    cond_multitest = np.zeros(function_enumeration_len, dtype=bool)
    cond_filter = np.ones(function_enumeration_len, dtype=bool)
    cond_PMIDs = np.zeros(function_enumeration_len, dtype=bool)
    cond_terms_reduced_with_ontology = np.zeros(function_enumeration_len, dtype=bool)
    background_ids_arr_of_string = np.empty(shape=(function_enumeration_len,), dtype=object)
    effectSizes = np.empty(function_enumeration_len, dtype=np.dtype("float64"))
    effectSizes.fill(np.nan)
    over_under_int_arr = np.zeros(function_enumeration_len, dtype=np.dtype("uint32")) # encoding of 1: "overrepresented", 2: "underrepresented", 0: "NaN"
    over_under_arr_of_string = np.empty(function_enumeration_len, np.dtype("U1"))
    return foreground_ids_arr_of_string, background_ids_arr_of_string, funcEnum_count_foreground, funcEnum_count_background, p_values, p_values_corrected, cond_multitest, blacklisted_terms_bool_arr_temp, cond_terms_reduced_with_ontology, cond_filter, cond_PMIDs, effectSizes, over_under_int_arr, over_under_arr_of_string

@boundscheck(False)
@wraparound(False)
cdef filter_parents_if_same_foreground(uint8[::1] blacklisted_terms_bool_arr_temp,
                                       cond_terms_reduced_with_ontology,
                                       dict lineage_dict_enum,
                                       df):
    """    
    potential speed up using C++ types for sets, BUT data is copied so profile 

    # distutils: language = c++    
    from libcpp.vector cimport vector
    from libcpp.set cimport set 
    """
    cdef:
        unsigned int term_enum, lineage_term
        # unsigned int lineage

    for group_terms in df.sort_values(["foreground_ids", "p_value", "hierarchical_level"], ascending=[True, True, False]).groupby("foreground_ids", sort=False).apply(lambda group: group["term_enum"].values):
        group_terms_set = set(group_terms)
        for term_enum in group_terms:
            if blacklisted_terms_bool_arr_temp[term_enum] == 0: # False
                cond_terms_reduced_with_ontology[term_enum] = True
                try:
                    lineage = lineage_dict_enum[term_enum] & group_terms_set # bitwise intersection
                except KeyError: # not in hierarchy (even though it should be, but some Reactome terms are inconsistent)
                    blacklisted_terms_bool_arr_temp[term_enum] = 1 # True
                    continue
                for lineage_term in lineage:
                    blacklisted_terms_bool_arr_temp[lineage_term] = 1 # True

def multiple_testing_per_entity_type(cond_etype, cond_multitest, p_values, p_values_corrected, indices_arr, num_total_tests):
    # select indices for given entity type and if multiple testing needs to be applied
    cond = cond_etype & cond_multitest
    # select p_values for BenjaminiHochberg
    p_values_2_BH = p_values[cond]
    # previously: num_total_tests = p_values_2_BH.shape[0]
    # select indices for BH
    indices_2_BH = indices_arr[cond]
    # sort p_values and remember indices sort order
    p_values_2_BH_sort_order = np.argsort(p_values_2_BH) # index positions of a reduced set
    indices_2_BH_of_superset = indices_2_BH[p_values_2_BH_sort_order]
    BenjaminiHochberg_cy(p_values, num_total_tests, p_values_corrected, indices_2_BH_of_superset)

def get_s_value(df, p_value_cutoff=0.05, KS_stat_cutoff=0.1, diff_proportions_cutoff=0.1):
    """
    calculate 's-value' type statistic in order to rank based on a combination of p-value and effect size
    for etypes -20, -25, and -26 (GOCC, BTO, and DOID) --> Common Language Effect Size
    for other etypes difference in ratios
    justification for cles_cutoff --> Kerby (https://doi.org/10.2466%2F11.IT.3.1) if the null is true the CLES is 50%
    justification for diff_proportions_cutoff --> unsure how to justify from lit. need be smaller than cles_cutoff
    --> changed from cles to KS_stat
    """
    min_pval = df[cn.p_value][df[cn.p_value] > 0].min()
    df["p_value_minlog"] = df[cn.p_value].apply(log_take_min_if_zero, args=(min_pval, ))
    df[cn.s_value] = 0.0
    # cond_scores = df[cn.etype].isin([-20, -25, -26])
    # p_value_cutoff = -1 * math.log10(p_value_cutoff) # test for values smaller than 0
    df[cn.s_value] = df["p_value_minlog"] * df[cn.effect_size]
    # df = df.drop(columns=["p_value_minlog"])
    return df[cn.s_value]

def log_take_min_if_zero(val, min_pval):
    try:
        return -1*math.log10(val)
    except:
        return -1*math.log10(min_pval)

def limit_to_entity_types(limit_2_entity_type, function_enumeration_len, etype_cond_dict, funcEnum_count_foreground):
    if limit_2_entity_type is not None:
        cond_limit_2_entity_type = np.zeros(function_enumeration_len, dtype=bool)
        for cond_name in ["cond_" + etype[1:] for etype in limit_2_entity_type.split(";")]:
            try:
                cond_limit_2_entity_type |= etype_cond_dict[cond_name] # add other etypes
            except KeyError: # user provided etype can be mistyped of non-existent
                pass
        # set funcEnumAssociations to zero where cond_limit_2_entity_type is False
        funcEnum_count_foreground[~cond_limit_2_entity_type] = 0
        return cond_limit_2_entity_type # return bool arr of locations that should NOT be tested
    else:
        return np.ones(function_enumeration_len, dtype=bool)

def limit_to_go_subset(etype_cond_dict, go_slim_subset, goslimtype_2_cond_dict, funcEnum_count_foreground):
    if go_slim_subset is None:
        return funcEnum_count_foreground
    cond_GO_etypes = etype_cond_dict["cond_21"] | etype_cond_dict["cond_22"] | etype_cond_dict["cond_23"]
    cond = cond_GO_etypes != goslimtype_2_cond_dict[go_slim_subset] # select all GO terms that are not slim
    # set these to count 0
    funcEnum_count_foreground[cond] = 0
    return funcEnum_count_foreground

def add_funcEnums_2_dict(protein_ans_fg, ENSP_2_functionEnumArray_dict, ENSP_2_tuple_funcEnum_score_dict):
    ### add Protein 2 functionEnum info for JensenLabScore data to get foregroundIDs in DF
    for protein in protein_ans_fg:
        try: # sort is probably not necessary # potential speedup removing the sorting
            ENSP_2_functionEnumArray_dict[protein] = np.sort(np.concatenate((ENSP_2_tuple_funcEnum_score_dict[protein][0], ENSP_2_functionEnumArray_dict[protein])))
        except KeyError:
            pass # print("protein {} not in ENSP_2_tuple_funcEnum_score_dict".format(protein)) # --> simply not annotated with anything from textmining

def add_funcEnums_2_dict_CSC(protein_AN_set, ENSP_2_functionEnumArray_dict, ENSP_2_rowIndex_dict, CSR_ENSPencoding_2_FuncEnum):
    """
    rowIndex = ENSP_2_rowIndex_dict["128UP_DROME"]
    CSR_ENSPencoding_2_FuncEnum[rowIndex].indices # --> FunEnums_array == ENSP_2_tuple_funcEnum_score_dict["128UP_DROME"][0]
    CSR_ENSPencoding_2_FuncEnum[rowIndex].data # --> Scores_array == ENSP_2_tuple_funcEnum_score_dict[ensp][1]
    """
    for protein in protein_AN_set:
        try:
            rowIndex = ENSP_2_rowIndex_dict[protein]
        except KeyError:
            continue
        funcEnum_array = CSR_ENSPencoding_2_FuncEnum[rowIndex].indices
        ENSP_2_functionEnumArray_dict[protein] = np.sort(np.concatenate((funcEnum_array, ENSP_2_functionEnumArray_dict[protein])))

def replace_secondary_and_primary_IDs(ans_string, secondary_2_primary_dict, invert_dict=False):
    if invert_dict:
        dict_2_use = {v: k for k, v in secondary_2_primary_dict.items()}
    else:
        dict_2_use = secondary_2_primary_dict
    ids_2_return = []
    for id_ in ans_string.split(";"): # if proteinGroup
        if id_ in dict_2_use:
            ids_2_return.append(dict_2_use[id_])
        else:
            ids_2_return.append(id_)
    return ";".join(ids_2_return)

def add_protein_groups_to_ENSP_2_functionEnumArray_dict(ENSP_2_functionEnumArray_dict, all_unique_proteinGroups):
    """
    for all protein groups
    """
    for proteinGroup in all_unique_proteinGroups:
        if proteinGroup not in ENSP_2_functionEnumArray_dict:
            functionEnumArray_list = []
            for protein in proteinGroup.split(";"):
                try:
                    functionEnumArray_list.append(ENSP_2_functionEnumArray_dict[protein])
                except KeyError: # no functional annotation for given protein
                    pass
            try:
                ENSP_2_functionEnumArray_dict[proteinGroup] = reduce(np.union1d, functionEnumArray_list)
            except TypeError: # empty list
                #ENSP_2_functionEnumArray_dict[proteinGroup] = False #np.array(dtype=np.dtype("uint32"))
                pass
    return ENSP_2_functionEnumArray_dict

def filter_stuff(args_dict, protein_ans_fg, p_values_corrected, foreground_ids_arr_of_string, funcEnum_count_foreground, year_arr, p_values, indices_arr, ENSP_2_functionEnumArray_dict, cond_filter, etype_cond_dict, cond_PMIDs, cond_etypes_with_ontology, cond_etypes_rem_foreground_ids, over_under_int_arr):
    FDR_cutoff, p_value_cutoff = args_dict["FDR_cutoff"], args_dict["p_value_cutoff"]
    cond_filter = (p_values_corrected <= FDR_cutoff) & (p_values <= p_value_cutoff)
    ### remove terms with only a single annotation
    if args_dict["filter_foreground_count_one"] is True:
        cond_filter &= funcEnum_count_foreground > 1
    else:  # remove terms without any annotation
        cond_filter &= funcEnum_count_foreground > 0

    ### overrepresented/underrepresented/both
    o_or_u_or_both_encoding = args_dict["o_or_u_or_both_encoding"]
    if o_or_u_or_both_encoding == 1: # overrepresented
        cond_o_or_u_or_both = over_under_int_arr == 1
    elif o_or_u_or_both_encoding == 2: # underrepresented
        cond_o_or_u_or_both = over_under_int_arr == 2
    elif o_or_u_or_both_encoding == 0: # both
        cond_o_or_u_or_both = over_under_int_arr > 0
    else:
        pass # should not happen
    cond_filter &= cond_o_or_u_or_both
    filter_PMID_top_n = args_dict["filter_PMID_top_n"]
    if filter_PMID_top_n is not None:
        cond_PMID_2_filter = cond_filter & etype_cond_dict["cond_56"]  # -56
        df_PMID = pd.DataFrame({"foreground_count": funcEnum_count_foreground[cond_PMID_2_filter].view(), "year": year_arr[cond_PMID_2_filter].view(), "p_value": p_values[cond_PMID_2_filter].view(), "FDR": p_values_corrected[cond_PMID_2_filter].view(), "indices_arr": indices_arr[cond_PMID_2_filter].view()})
        indices_PMID = df_PMID.sort_values(["FDR", "p_value", "year", "foreground_count"], ascending=[True, True, False, False])["indices_arr"].values[:filter_PMID_top_n]
        for index_ in indices_PMID:
            cond_PMIDs[index_] = True
    else:  # since no filtering use all PMIDs
        cond_PMIDs = cond_filter & etype_cond_dict["cond_56"]
    cond_etypes_with_ontology_filtered = cond_etypes_with_ontology & cond_filter  # {-21, -22, -23, -51, -57}
    # entity_types_with_ontology = {-20, -21, -22, -23, -25, -26, -51, -57} # Interpro has ontology, but omitted here to turn off filter_parents functionality
    cond_etypes_rem_foreground_ids_filtered = cond_etypes_rem_foreground_ids & cond_filter  # remaining etypes -52, -53, -54, -55
    cond_IDs_2_query = (cond_PMIDs | cond_etypes_with_ontology_filtered | cond_etypes_rem_foreground_ids_filtered)
    ### get foreground IDs of relevant subset --> array for entire data set
    ## exclude TextMining KS functionEnumerations since these are probably not very informative and we need performance --> don't exclude
#     if not KS_etypes_FG_IDs:
#         cond_IDs_2_query = cond_IDs_2_query & ~cond_KS_etypes # commented on purpose since STRING needs these
    funcEnum_indices_for_IDs = indices_arr[cond_IDs_2_query]
    foreground_ids_arr_of_string = map_funcEnum_2_ENSPs(protein_ans_fg, ENSP_2_functionEnumArray_dict, funcEnum_indices_for_IDs, foreground_ids_arr_of_string)
#     if not KS_etypes_FG_IDs:
#         foreground_ids_arr_of_string[cond_KS_etypes] = "" # commented out for STRING
    return foreground_ids_arr_of_string, funcEnum_indices_for_IDs, cond_etypes_with_ontology_filtered, cond_etypes_rem_foreground_ids_filtered, cond_filter

def run_characterize_foreground_cy(ui, preloaded_objects_per_analysis, static_preloaded_objects, low_memory=False):
    if not low_memory:
        ENSP_2_functionEnumArray_dict, year_arr, hierlevel_arr, entitytype_arr, functionalterm_arr, indices_arr, description_arr, category_arr, etype_2_minmax_funcEnum, function_enumeration_len, etype_cond_dict, etype_2_num_functions_dict, taxid_2_proteome_count, taxid_2_tuple_funcEnum_index_2_associations_counts, lineage_dict_enum, blacklisted_terms_bool_arr, cond_etypes_with_ontology, cond_etypes_rem_foreground_ids, kegg_taxid_2_acronym_dict, goslimtype_2_cond_dict = static_preloaded_objects
    else:  # missing: ENSP_2_functionEnumArray_dict
        year_arr, hierlevel_arr, entitytype_arr, functionalterm_arr, indices_arr, description_arr, category_arr, etype_2_minmax_funcEnum, function_enumeration_len, etype_cond_dict, etype_2_num_functions_dict, taxid_2_proteome_count, taxid_2_tuple_funcEnum_index_2_associations_counts, lineage_dict_enum, blacklisted_terms_bool_arr, cond_etypes_with_ontology, cond_etypes_rem_foreground_ids, kegg_taxid_2_acronym_dict, goslimtype_2_cond_dict = static_preloaded_objects
    foreground_ids_arr_of_string, background_ids_arr_of_string, funcEnum_count_foreground, funcEnum_count_background, p_values, p_values_corrected, cond_multitest, blacklisted_terms_bool_arr_temp, cond_terms_reduced_with_ontology, cond_filter, cond_PMIDs, effectSizes, over_under_int_arr, over_under_arr_of_string = preloaded_objects_per_analysis
    em = ui.enrichment_method
    foreground_n = ui.get_foreground_n()
    args_dict = ui.args_dict
    filter_foreground_count_one = args_dict["filter_foreground_count_one"]

    protein_ans_fg = ui.get_foreground_an_set()
    if low_memory:
        ENSP_2_functionEnumArray_dict = query.get_functionEnumArray_from_proteins(ui.get_all_individual_AN(), dict_2_array=True)
    ### add protein groups to ENSP_2_functionEnumArray_dict
    ENSP_2_functionEnumArray_dict = add_protein_groups_to_ENSP_2_functionEnumArray_dict(ENSP_2_functionEnumArray_dict, ui.get_all_unique_proteinGroups())

    ## count foreground
    count_all_terms(ENSP_2_functionEnumArray_dict, protein_ans_fg, funcEnum_count_foreground)

    ## limit to given entity types
    cond_limit_2_entity_type = limit_to_entity_types(args_dict["limit_2_entity_type"], function_enumeration_len, etype_cond_dict, funcEnum_count_foreground)
    limit_to_go_subset(etype_cond_dict, args_dict["go_slim_subset"], goslimtype_2_cond_dict, funcEnum_count_foreground)

    ### calc ratio in foreground, count foreground / len(protein_ans)
    ratio_in_foreground = funcEnum_count_foreground / foreground_n

    ### concatenate filtered results
    if filter_foreground_count_one:
        cond_2_return = funcEnum_count_foreground > 1
    else:
        cond_2_return = funcEnum_count_foreground >= 1

    ### limit PMID results
    filter_PMID_top_n = args_dict["filter_PMID_top_n"]
    if filter_PMID_top_n is not None:
        cond_PMID_2_filter = cond_2_return & etype_cond_dict["cond_56"]
        df_PMID = pd.DataFrame({"foreground_count": funcEnum_count_foreground[cond_PMID_2_filter].view(), "year": year_arr[cond_PMID_2_filter].view(), "indices_arr": indices_arr[cond_PMID_2_filter].view()})
        indices_PMID = df_PMID.sort_values(["foreground_count", "year"], ascending=[False, False])["indices_arr"].values[:filter_PMID_top_n]
        # set all PMIDs to False and then include only those that were selected
        cond_2_return[etype_cond_dict["cond_56"]] = False
        for index_ in indices_PMID:
            cond_2_return[index_] = True
    cond_2_return[blacklisted_terms_bool_arr > 0] = False

    try:
        privileged = args_dict["privileged"]
    except KeyError:
        privileged = False
    if not privileged:
        # remove KEGG unless privileged
        cond_kegg = etype_cond_dict["cond_52"]
        cond_2_return = cond_2_return & ~cond_kegg

    funcEnum_indices_for_IDs = indices_arr[cond_2_return]
    foreground_ids_arr_of_string = map_funcEnum_2_ENSPs(protein_ans_fg, ENSP_2_functionEnumArray_dict, funcEnum_indices_for_IDs, foreground_ids_arr_of_string)
    if not low_memory:
        df_2_return = pd.DataFrame({cn.term: functionalterm_arr[cond_2_return].view(),
                                    cn.hierarchical_level: hierlevel_arr[cond_2_return].view(),
                                    cn.category: category_arr[cond_2_return].view(),
                                    cn.etype: entitytype_arr[cond_2_return].view(),
                                    cn.description: description_arr[cond_2_return].view(),
                                    cn.year: year_arr[cond_2_return].view(),
                                    cn.ratio_in_FG: ratio_in_foreground[cond_2_return].view(),
                                    cn.FG_IDs: foreground_ids_arr_of_string[cond_2_return].view(),
                                    cn.FG_count: funcEnum_count_foreground[cond_2_return].view()})
    else:
        df_2_return = pd.DataFrame({cn.term: functionalterm_arr[cond_2_return].view(),
                                    cn.hierarchical_level: hierlevel_arr[cond_2_return].view(),
                                    cn.etype: entitytype_arr[cond_2_return].view(),
                                    cn.year: year_arr[cond_2_return].view(),
                                    cn.ratio_in_FG: ratio_in_foreground[cond_2_return].view(),
                                    cn.FG_IDs: foreground_ids_arr_of_string[cond_2_return].view(),
                                    cn.FG_count: funcEnum_count_foreground[cond_2_return].view(),
                                    cn.funcEnum: indices_arr[cond_2_return].view()})
        df_2_return[cn.category] = df_2_return[cn.etype].apply(lambda etype: variables.entityType_2_functionType_dict[etype])
        funcEnum_2_description_dict = query.get_function_description_from_funcEnum(indices_arr[cond_2_return].tolist())
        df_2_return[cn.description] = df_2_return[cn.funcEnum].apply(lambda funcEnum: funcEnum_2_description_dict[funcEnum])
    df_2_return = ui.translate_primary_back_to_secondary(df_2_return)
    df_2_return[cn.FG_n] = foreground_n
    ### rank everything correctly except PMIDs, "year"-column will only affect PMIDs
    df_2_return = df_2_return.sort_values([cn.etype, cn.year, cn.FG_count], ascending=[False, False, False]).reset_index(drop=True)
    # debug delete me --> df_2_return = df_2_return.sort_values([cn.etype, "s_value_abs", cn.hierarchical_level, cn.year], ascending=[False, False, False, False])

    if args_dict["STRING_beta"]:
        # return df_2_return[list(cn.cols_sort_order_characterize_foreground_STRING_beta)].rename(columns=cn.colnames_2_rename_dict_STRING_beta)
        return df_2_return.rename(columns=cn.colnames_2_rename_dict_STRING_beta)[list(cn.cols_sort_order_characterize_foreground_STRING_beta)]
    elif args_dict["STRING_API"]:
        df_2_return["ncbiTaxonId"] = args_dict["taxid"]
        df_2_return["preferredNames"] = ""
        return df_2_return.rename(columns=cn.colnames_2_rename_dict_STRING_API)[list(cn.cols_sort_order_STRING_API_functional_annotation)]
    cond_PMIDs = df_2_return[cn.etype] == -56
    df_2_return.loc[~cond_PMIDs, cn.rank] = df_2_return[~cond_PMIDs].groupby(cn.etype)[cn.FG_count].rank(ascending=False, method="first").fillna(value=df_2_return.shape[0])
    df_2_return.loc[cond_PMIDs, cn.rank] = df_2_return[cond_PMIDs].groupby(cn.etype)[cn.year].rank(ascending=False, method="first").fillna(value=df_2_return.shape[0])
    df_2_return[cn.rank] = df_2_return[cn.rank].astype(int)
    return df_2_return[list(cn.cols_2_return_run_characterize_foreground_cy)]


## run ze function!

In [25]:
from importlib import reload
reload(userinput)
ENSP_2_tuple_funcEnum_score_dict = None

contiguous = True
foreground_n = 20
foreground_input = sorted(get_random_human_ENSP(foreground_n, joined_for_web=False, contiguous=contiguous, UniProt_ID=True))
# foreground_input = ['MEF2A_HUMAN', 'MEF2B_HUMAN', 'MEF2C_HUMAN', 'MEF2D_HUMAN', 'MEFV_HUMAN', 'MEG10_HUMAN', 'MEG11_HUMAN', 'MEGF6_HUMAN', 'MEGF8_HUMAN', 'MEGF9_HUMAN', 'MEI1_HUMAN', 'MEI4_HUMAN', 'MEIG1_HUMAN', 'MEIKN_HUMAN', 'MEIOB_HUMAN', 'MEIOC_HUMAN', 'MEIS1_HUMAN', 'MEIS2_HUMAN', 'MEIS3_HUMAN', 'MELK_HUMAN']
from_file = False # read user input from file
fn_userinput = r"/Users/dblyon/modules/cpr/agotool/data/exampledata/Example_Yeast_acetylation_abundance_correction.txt"
# fn_userinput = r"/Users/dblyon/Downloads/agotoolquestions/ClpP2up_KEimputed_aGOtool.txt"
# fn_userinput = r"/Users/dblyon/modules/cpr/agotool/data/exampledata/Example_1_Yeast_acetylation_foreground_only.txt"
# fn_userinput = r"/Users/dblyon/modules/cpr/agotool/data/exampledata/Example_1.1_Yeast_acetylation_without_abundance.txt"

enrichment_method = "compare_samples" # "" "abundance_correction" "compare_samples" "genome" "compare_groups"
args_dict = {}
args_dict["enrichment_method"] = enrichment_method
args_dict["taxid"] = 9606 # 9606 # 559292 Yeast
args_dict["FDR_cutoff"] = 0.05
args_dict["p_value_cutoff"] = 0.01
args_dict["limit_2_entity_type"] = None # "-20;-25;-26" #"-21;-22;-23;-51;-52;-53;-54;-55;-56-57;-58"
args_dict["filter_PMID_top_n"] = 100
args_dict["filter_foreground_count_one"] = True
args_dict["filter_parents"] = True
args_dict["go_slim_subset"] = None # "generic"
args_dict["o_or_u_or_both"] = "both" # "both" "underrepresented" "overrepresented"
args_dict["multiple_testing_per_etype"] = True
args_dict["privileged"] = True
args_dict["score_cutoff"] = 0
taxid = args_dict["taxid"]
debug = False
profile = False
simplified_output = False
args_dict["simplified_output"] = simplified_output
args_dict["STRING_beta"] = False
# args_dict["multiple_testing_stringency"] = "A"
background_n = 300
contiguous = True
# background_input = sorted(get_random_human_ENSP(background_n, joined_for_web=False, contiguous=contiguous)) # background_input = ENSPs_homo
# background_input = query.get_proteins_of_taxid(taxid, read_from_flat_files=True)
# background_input = None

### debug
fn = r"/Users/dblyon/Downloads/res.tsv"
df = pd.read_csv(fn, sep='\t')
fg = []
for arr in df["foreground_ids"].unique():
    fg += arr.split(";")
foreground_input = sorted(set(fg))

bg = []
for arr in df["background_ids"].unique():
    bg += arr.split(";")    
background_input = sorted(set(bg))
### debug


if from_file:
    ui = userinput.Userinput(pqo, fn_userinput, args_dict=args_dict)
else:
    ui = userinput.Userinput(pqo, fn=None, foreground_string=userinput.stringify_for_Userinput(foreground_input), background_string=userinput.stringify_for_Userinput(background_input), args_dict=args_dict)
ncbi = pqo.ncbi
preloaded_objects_per_analysis = pqo.get_preloaded_objects_per_analysis()

if profile:
    profile = line_profiler.LineProfiler(calc_pvalues_v2) # run_enrichment_cy, KolmogorovSmirnov_sparse_cy, KolmogorovSmirnov_sparse_cy_genome
    profile.runcall(run_enrichment_cy, ENSP_2_tuple_funcEnum_score_dict, ncbi, ui, preloaded_objects_per_analysis, static_preloaded_objects, low_memory=True, debug=debug, KS_method=KS_method)
    profile.print_stats()
else:
    if enrichment_method == "characterize_foreground":
        df = run_characterize_foreground_cy(ui, preloaded_objects_per_analysis, static_preloaded_objects, low_memory=True)
        df = df.sort_values(["etype", "FG_count"], ascending=[False, False])
    else:
        if debug:
            df = run_enrichment_cy(ncbi, ui, preloaded_objects_per_analysis, static_preloaded_objects, low_memory=True, debug=debug)
        else:
            df = run_enrichment_cy(ncbi, ui, preloaded_objects_per_analysis, static_preloaded_objects, low_memory=True, debug=debug)
#             funcEnum_count_foreground, funcEnum_count_background, foreground_n, background_n, p_values, cond_multitest, effectSizes, over_under_int_arr, o_or_u_or_both_encoding = run_enrichment_cy(ncbi, ui, preloaded_objects_per_analysis, static_preloaded_objects, low_memory=True, debug=debug, pvalue_version="debug")
            df = df.sort_values(["etype", "rank"], ascending=[False, True])
            print(df.shape)
            print(df.groupby("category")["term"].count())

(13, 21)
category
Gene Ontology biological process               1
Gene Ontology cellular component               2
Gene Ontology cellular component TEXTMINING    3
Reactome                                       1
UniProt keywords                               6
Name: term, dtype: int64


In [27]:
df[["term", "category", "etype"]]

,term,category,etype
0,GOCC:0005576,Gene Ontology cellular component TEXTMINING,-20
1,GOCC:0031012,Gene Ontology cellular component TEXTMINING,-20
2,GOCC:0062023,Gene Ontology cellular component TEXTMINING,-20
3,GO:0030198,Gene Ontology biological process,-21
4,GO:0031012,Gene Ontology cellular component,-22
5,GO:0062023,Gene Ontology cellular component,-22
6,KW-0964,UniProt keywords,-51
7,KW-0732,UniProt keywords,-51
8,KW-1015,UniProt keywords,-51
9,KW-0325,UniProt keywords,-51


In [34]:
import requests
from io import StringIO

url_ = r"https://agotool.org/api_orig"

fg = "%0d".join(foreground_input)
bg = "%0d".join(background_input)
result = requests.post(url_,
                   params={"output_format": "tsv",
                           "enrichment_method": "compare_samples"},
                   data={"foreground": fg, "background": bg})
df = pd.read_csv(StringIO(result.text), sep='\t')
df.head()

,term,hierarchical_level,description,year,over_under,p_value,FDR,effect_size,ratio_in_foreground,ratio_in_background,...,foreground_n,background_count,background_n,foreground_ids,background_ids,s_value,rank,funcEnum,category,etype
0,GOCC:0005576,2,Extracellular region,-1,o,1.362150e-06,0.002672,0.338243,0.629032,0.290789,...,62,221,760,Q15582;P07858;P43235;Q9Y646;O00622;Q9Y240;P024...,Q13685;P60709;P63261;P06280;Q09666;P15121;P151...,1.984056,1,209,Gene Ontology cellular component TEXTMINING,-20
1,GOCC:0031012,4,Extracellular matrix,-1,o,2.357761e-07,0.000925,0.218930,0.274194,0.055263,...,62,42,760,Q15582;O00622;P02452;P08123;P27658;Q9Y6C2;O959...,P08758;Q9UHI8;Q15582;O00622;P48509;P41208;P024...,1.450961,2,1023,Gene Ontology cellular component TEXTMINING,-20
2,GOCC:0062023,5,Collagen-containing extracellular matrix,-1,o,1.417667e-06,0.002672,0.193251,0.241935,0.048684,...,62,37,760,Q15582;O00622;P02452;P08123;P27658;Q9Y6C2;O959...,P08758;Q15582;O00622;P48509;P41208;P02452;P081...,1.130216,3,2299,Gene Ontology cellular component TEXTMINING,-20
3,GO:0030198,5,extracellular matrix organization,-1,o,2.963909e-07,0.005894,0.251528,0.322581,0.071053,...,62,54,760,Q15582;P07384;P43235;O00622;P02452;P08123;P276...,Q9UHI8;Q15582;P07384;P43235;P07711;Q03135;O006...,1.642009,1,9384,Gene Ontology biological process,-21
4,GO:0030312,4,external encapsulating structure,-1,o,2.828384e-07,0.000781,0.284126,0.370968,0.086842,...,62,66,760,Q15582;P07858;O00622;P02452;P08123;P27658;Q6UV...,P04083;P08758;Q9UHI8;Q15582;P27797;P07858;P536...,1.860586,1,24609,Gene Ontology cellular component,-22


In [35]:
df

,term,hierarchical_level,description,year,over_under,p_value,FDR,effect_size,ratio_in_foreground,ratio_in_background,...,foreground_n,background_count,background_n,foreground_ids,background_ids,s_value,rank,funcEnum,category,etype
0,GOCC:0005576,2,Extracellular region,-1,o,1.362150e-06,0.002672,0.338243,0.629032,0.290789,...,62,221,760,Q15582;P07858;P43235;Q9Y646;O00622;Q9Y240;P024...,Q13685;P60709;P63261;P06280;Q09666;P15121;P151...,1.984056,1,209,Gene Ontology cellular component TEXTMINING,-20
1,GOCC:0031012,4,Extracellular matrix,-1,o,2.357761e-07,0.000925,0.218930,0.274194,0.055263,...,62,42,760,Q15582;O00622;P02452;P08123;P27658;Q9Y6C2;O959...,P08758;Q9UHI8;Q15582;O00622;P48509;P41208;P024...,1.450961,2,1023,Gene Ontology cellular component TEXTMINING,-20
2,GOCC:0062023,5,Collagen-containing extracellular matrix,-1,o,1.417667e-06,0.002672,0.193251,0.241935,0.048684,...,62,37,760,Q15582;O00622;P02452;P08123;P27658;Q9Y6C2;O959...,P08758;Q15582;O00622;P48509;P41208;P02452;P081...,1.130216,3,2299,Gene Ontology cellular component TEXTMINING,-20
3,GO:0030198,5,extracellular matrix organization,-1,o,2.963909e-07,0.005894,0.251528,0.322581,0.071053,...,62,54,760,Q15582;P07384;P43235;O00622;P02452;P08123;P276...,Q9UHI8;Q15582;P07384;P43235;P07711;Q03135;O006...,1.642009,1,9384,Gene Ontology biological process,-21
4,GO:0030312,4,external encapsulating structure,-1,o,2.828384e-07,0.000781,0.284126,0.370968,0.086842,...,62,66,760,Q15582;P07858;O00622;P02452;P08123;P27658;Q6UV...,P04083;P08758;Q9UHI8;Q15582;P27797;P07858;P536...,1.860586,1,24609,Gene Ontology cellular component,-22
5,GO:0031012,4,extracellular matrix,-1,o,2.828384e-07,0.000781,0.284126,0.370968,0.086842,...,62,66,760,Q15582;P07858;O00622;P02452;P08123;P27658;Q6UV...,P04083;P08758;Q9UHI8;Q15582;P27797;P07858;P536...,1.860586,2,24690,Gene Ontology cellular component,-22
6,GO:0062023,5,collagen-containing extracellular matrix,-1,o,1.149455e-06,0.001058,0.261078,0.338710,0.077632,...,62,59,760,Q15582;P07858;O00622;P02452;P08123;P27658;Q6UV...,P04083;P08758;Q9UHI8;Q15582;P27797;P07858;P536...,1.550675,3,25722,Gene Ontology cellular component,-22
7,KW-0964,2,Secreted,-1,o,8.744909e-07,0.001026,0.330560,0.451613,0.121053,...,62,92,760,Q15582;P07858;P43235;Q9Y646;O00622;Q9Y240;P024...,Q9H4A4;P04083;Q9UHI8;P61769;Q15582;Q07021;P277...,2.002615,1,46784,UniProt keywords,-51
8,KW-0732,2,Signal,-1,o,1.171856e-04,0.034365,0.233531,0.467742,0.234211,...,62,178,760,Q15582;P07858;P43235;Q9Y646;O00622;Q9Y240;P024...,P06280;P30533;P20933;Q9UHI8;P61769;Q15582;P110...,0.918041,2,46569,UniProt keywords,-51
9,KW-1015,2,Disulfide bond,-1,o,1.171856e-04,0.034365,0.233531,0.467742,0.234211,...,62,178,760,Q15582;P07858;P43235;O00622;Q9Y240;P02452;P081...,P06280;P15144;P04083;P27695;P20933;Q9UHI8;P617...,0.918041,3,46835,UniProt keywords,-51


In [22]:
from importlib import reload
reload(userinput)
ENSP_2_tuple_funcEnum_score_dict = None
#######################
### examples
# foreground_input = ["HBA_HUMAN", "HBB_HUMAN", "HBD_HUMAN", "HBE_HUMAN"] # example 1: Human hemoglobin, genome
# example 2: Yeast acetylation, abundance_correction. Example_1_Yeast_acetylation_abundance_correction.txt
# example 3: compare_samples
# human plasma liver cirrhosis Lili Niu
# foreground_input = ['P05062;A0A024R145;A8K430;A0A087WXX2;Q8NHT3', 'Q08380;B4DVE1;A0A0S2Z3Y1;B3KP88;B4DDG4;B4DWA8;B4DI70', 'P04004;D9ZGG2;B7Z553', 'O95445', 'P01833', 'P43652', 'Q5SRP5']
# mouse Insulin (STRING network )
# foreground_input = ['10090.ENSMUSP00000005671', '10090.ENSMUSP00000020846', '10090.ENSMUSP00000022921', '10090.ENSMUSP00000028252', '10090.ENSMUSP00000056668', '10090.ENSMUSP00000061877', '10090.ENSMUSP00000084464', '10090.ENSMUSP00000088837', '10090.ENSMUSP00000099787', '10090.ENSMUSP00000099862', '10090.ENSMUSP00000104298']
# mouse interferon (STRING network)
# foreground_input = ['10090.ENSMUSP00000001036', '10090.ENSMUSP00000023689', '10090.ENSMUSP00000023693', '10090.ENSMUSP00000038121', '10090.ENSMUSP00000056720', '10090.ENSMUSP00000066743', '10090.ENSMUSP00000092581', '10090.ENSMUSP00000099842', '10090.ENSMUSP00000100872', '10090.ENSMUSP00000120525', '10090.ENSMUSP00000127921']
#Q9R117\nP33896\nO35664\nO35716\nP01575\nP42225\nP07351\nP52332\nQ9WVL2\nQ61179\nQ61716
# foreground_input = ["ADH1_YEAST", "PDC1_YEAST", "PFKA1_YEAST"]
# foreground_input = ["PGM1_YEAST", "G6PI_YEAST", "PMG2_YEAST", "CISY2_YEAST"]
############
contiguous = True
foreground_n = 300
# foreground_input = sorted(get_random_human_ENSP(foreground_n, joined_for_web=False, contiguous=contiguous, UniProt_ID=True))
# foreground_input = ['MEF2A_HUMAN', 'MEF2B_HUMAN', 'MEF2C_HUMAN', 'MEF2D_HUMAN', 'MEFV_HUMAN', 'MEG10_HUMAN', 'MEG11_HUMAN', 'MEGF6_HUMAN', 'MEGF8_HUMAN', 'MEGF9_HUMAN', 'MEI1_HUMAN', 'MEI4_HUMAN', 'MEIG1_HUMAN', 'MEIKN_HUMAN', 'MEIOB_HUMAN', 'MEIOC_HUMAN', 'MEIS1_HUMAN', 'MEIS2_HUMAN', 'MEIS3_HUMAN', 'MELK_HUMAN']
from_file = True # read user input from file
fn_userinput = r"/Users/dblyon/modules/cpr/agotool/data/exampledata/Example_Yeast_acetylation_abundance_correction.txt"
# fn_userinput = r"/Users/dblyon/Downloads/agotoolquestions/ClpP2up_KEimputed_aGOtool.txt"
# fn_userinput = r"/Users/dblyon/modules/cpr/agotool/data/exampledata/Example_1_Yeast_acetylation_foreground_only.txt"
# fn_userinput = r"/Users/dblyon/modules/cpr/agotool/data/exampledata/Example_1.1_Yeast_acetylation_without_abundance.txt"
# foreground_input = ["P69905"]
# foreground_input = corona

enrichment_method = "compare_samples" # "" "abundance_correction" "compare_samples" "genome"
args_dict = {}
args_dict["enrichment_method"] = enrichment_method
args_dict["taxid"] = 9606 # 9606 # 559292 Yeast
args_dict["FDR_cutoff"] = 0.05
args_dict["p_value_cutoff"] = 0.01
args_dict["limit_2_entity_type"] = None # "-20;-25;-26" #"-21;-22;-23;-51;-52;-53;-54;-55;-56-57;-58"
args_dict["filter_PMID_top_n"] = 100
args_dict["filter_foreground_count_one"] = True
args_dict["filter_parents"] = True
args_dict["go_slim_subset"] = None # "generic"
args_dict["o_or_u_or_both"] = "both" # "both" "underrepresented" "overrepresented"
args_dict["multiple_testing_per_etype"] = True
args_dict["privileged"] = True
args_dict["score_cutoff"] = 0
# args_dict["foreground_replicates"] = 10
# args_dict["background_replicates"] = 10
taxid = args_dict["taxid"]
debug = False
profile = False
simplified_output = False
args_dict["simplified_output"] = simplified_output
args_dict["STRING_beta"] = False
# KS_method = "sparse_scipy" # {"scipy", "cy", "sparse_scipy"}
background_n = 300
contiguous = True
# background_input = sorted(get_random_human_ENSP(background_n, joined_for_web=False, contiguous=contiguous)) # background_input = ENSPs_homo
# background_input = query.get_proteins_of_taxid(taxid, read_from_flat_files=True)
# background_input = None
if from_file:
    ui = userinput.Userinput(pqo, fn_userinput, args_dict=args_dict)
else:
    ui = userinput.Userinput(pqo, fn=None, foreground_string=userinput.stringify_for_Userinput(foreground_input), background_string=userinput.stringify_for_Userinput(background_input), args_dict=args_dict)
ncbi = pqo.ncbi
preloaded_objects_per_analysis = pqo.get_preloaded_objects_per_analysis()

if profile:
    profile = line_profiler.LineProfiler(calc_pvalues_v2) # run_enrichment_cy, KolmogorovSmirnov_sparse_cy, KolmogorovSmirnov_sparse_cy_genome
    profile.runcall(run_enrichment_cy, ENSP_2_tuple_funcEnum_score_dict, ncbi, ui, preloaded_objects_per_analysis, static_preloaded_objects, low_memory=True, debug=debug, KS_method=KS_method)
    profile.print_stats()
else:
    if enrichment_method == "characterize_foreground":
        df = run_characterize_foreground_cy(ui, preloaded_objects_per_analysis, static_preloaded_objects, low_memory=True)
        df = df.sort_values(["etype", "FG_count"], ascending=[False, False])
    else:
        if debug:
            df = run_enrichment_cy(ncbi, ui, preloaded_objects_per_analysis, static_preloaded_objects, low_memory=True, debug=debug)
        else:
            df = run_enrichment_cy(ncbi, ui, preloaded_objects_per_analysis, static_preloaded_objects, low_memory=True, debug=debug)
#             funcEnum_count_foreground, funcEnum_count_background, foreground_n, background_n, p_values, cond_multitest, effectSizes, over_under_int_arr, o_or_u_or_both_encoding = run_enrichment_cy(ncbi, ui, preloaded_objects_per_analysis, static_preloaded_objects, low_memory=True, debug=debug, pvalue_version="debug")
            df = df.sort_values(["etype", "rank"], ascending=[False, True])
            print(df.shape)
            print(df.groupby("category")["term"].count())

(53, 20)
category
Gene Ontology cellular component               25
Gene Ontology cellular component TEXTMINING    16
UniProt keywords                               12
Name: term, dtype: int64


In [179]:
df_new = df.copy()
# df_new["effect_size"]

In [181]:
df_old = df.copy()

In [182]:
df_old.shape, df_new.shape

((56, 20), (56, 20))

In [183]:
df_old.groupby("category")["term"].count()

category
Gene Ontology cellular component               25
Gene Ontology cellular component TEXTMINING    19
UniProt keywords                               12
Name: term, dtype: int64

In [184]:
df_new.groupby("category")["term"].count()

category
Gene Ontology cellular component               25
Gene Ontology cellular component TEXTMINING    19
UniProt keywords                               12
Name: term, dtype: int64

In [185]:
df_old.tail()

,term,hierarchical_level,description,year,over_under,p_value,FDR,effect_size,ratio_in_foreground,ratio_in_background,foreground_count,foreground_n,background_count,background_n,foreground_ids,s_value,rank,funcEnum,category,etype
51,KW-0238,2,DNA-binding,-1,o,0.000237,0.021388,0.037964,0.082830,0.044866,96,1159,52,1159,P14164;Q02486;P40535;P07248;P41696;P22035;P171...,0.137640,8,46776,UniProt keywords,-51
52,KW-0698,2,rRNA processing,-1,o,0.000238,0.021388,0.036238,0.075928,0.039689,88,1159,46,1159,P40493;P25627;P33322;Q12389;P24783;P20447;P204...,0.131291,9,47186,UniProt keywords,-51
53,KW-0732,2,Signal,-1,u,0.000034,0.003653,-0.024159,0.007765,0.031924,9,1159,37,1159,P25371;P37302;P16474;Q06350;P43555;Q12408;P179...,-0.107883,10,47213,UniProt keywords,-51
54,KW-0156,2,Chromatin regulator,-1,o,0.000068,0.006653,0.025022,0.035375,0.010354,41,1159,12,1159,P80428;Q05123;P32447;Q07457;P32657;Q08923;P313...,0.104276,11,46701,UniProt keywords,-51
55,KW-0010,2,Activator,-1,o,0.000360,0.028194,0.025884,0.043141,0.017256,50,1159,20,1159,P14164;P07248;P80428;P22035;P14066;Q06697;P891...,0.089131,12,46569,UniProt keywords,-51


In [188]:
from pandas import testing as pd_testing
pd_testing.assert_frame_equal(df_old, df_new)

In [186]:
term = "KW-0010"
df_old[df_old["term"] == term].head()

,term,hierarchical_level,description,year,over_under,p_value,FDR,effect_size,ratio_in_foreground,ratio_in_background,foreground_count,foreground_n,background_count,background_n,foreground_ids,s_value,rank,funcEnum,category,etype
55,KW-0010,2,Activator,-1,o,0.00036,0.028194,0.025884,0.043141,0.017256,50,1159,20,1159,P14164;P07248;P80428;P22035;P14066;Q06697;P891...,0.089131,12,46569,UniProt keywords,-51


In [187]:
df_new[df_new["term"] == term].head()

,term,hierarchical_level,description,year,over_under,p_value,FDR,effect_size,ratio_in_foreground,ratio_in_background,foreground_count,foreground_n,background_count,background_n,foreground_ids,s_value,rank,funcEnum,category,etype
55,KW-0010,2,Activator,-1,o,0.00036,0.028194,0.025884,0.043141,0.017256,50,1159,20,1159,P14164;P07248;P80428;P22035;P14066;Q06697;P891...,0.089131,12,46569,UniProt keywords,-51


In [19]:
reload(userinput)

<module 'userinput' from '/Users/dblyon/modules/cpr/agotool/app/python/userinput.py'>

In [35]:
import random
taxid = random.choice(query.get_taxids())
background = query.get_proteins_of_taxid(taxid)
foreground = random.sample(background, 200)
intensity = [str(ele) for ele in np.random.normal(size=len(background))]

In [58]:
0.15 + 0.09722 + 0.0689 + 0.107 + 0.107

0.53012

In [ ]:
len(missingScores_list), missingScores_list

In [ ]:
scores_list_sparse, scores_list_ff

In [ ]:
fn_userinput = os.path.join(variables.EXAMPLE_FOLDER, "Example_1_Yeast_acetylation_abundance_correction.txt")
df = pd.read_csv(fn_userinput, sep='\t')
fg = df.loc[df["Foreground"].notnull(), "Foreground"].tolist()
d = list(query.map_secondary_2_primary_ANs(fg).values())

In [ ]:
uniprotid = 'YD21B_YEAST'
ENSP_2_tuple_funcEnum_score_dict["ARV1_ARATH"]

In [ ]:
dfs.to_csv("~/Downloads/Example_1_Yeast_acetylation_abundance_correction_OUTPUT_SCIPY.txt", sep="\t", header=True, index=False)
dfc.to_csv("~/Downloads/Example_1_Yeast_acetylation_abundance_correction_OUTPUT_CYTHON.txt", sep="\t", header=True, index=False)

In [ ]:
list(ENSP_2_tuple_funcEnum_score_dict.keys())[:3]

In [ ]:
print(len(fg), len(set(fg)))
fg[:3]

In [ ]:
# data = fg_scores_matrix_data
# indptr = fg_scores_matrix_indptr
# for i in range(len(indptr[:-1])):  # get column values
#     index_row_start = indptr[i]
#     index_row_stop = indptr[i + 1]
#     if index_row_start == index_row_stop:
#         continue
#     funcEnum = i
#     scores_list_sparse = sorted(data[index_row_start:index_row_stop])
#     scores_list_ff = sorted(funcEnum_2_scores_dict_fg[funcEnum])
#     assert len(scores_list_sparse) ==
#     len(scores_list_ff)
#     assert scores_list_sparse == scores_list_ff

In [ ]:
len(scores_list_sparse), len(scores_list_ff)

In [ ]:
funcEnum

In [ ]:
funcEnum = 256 # Nucleus
sum(funcEnum_2_scores_dict_fg[funcEnum] > 0)

In [ ]:
foreground_n, background_n

In [ ]:
# ui.foreground

In [ ]:
pqo.taxid_2_proteome_count[559292]

In [ ]:
print(dfc.groupby("etype")["term"].count())
print(dfs.groupby("etype")["term"].count())

In [ ]:
# dfc[dfc["etype"] == -20].head()

In [ ]:
dfs[dfs["term"] == "GOCC:0005634"].FG_IDs.values[0] == dfc[dfc["term"] == "GOCC:0005634"].FG_IDs.values[0]

In [ ]:
len(dfs[dfs["term"] == "GOCC:0005634"].FG_IDs.values[0].split(";")), len(dfc[dfc["term"] == "GOCC:0005634"].FG_IDs.values[0].split(";"))

In [ ]:
dfs[dfs["term"] == "GOCC:0005634"]

In [ ]:
dfc[dfc["term"] == "GOCC:0005634"]

In [ ]:
0.986195 - 0.950679

In [ ]:
dfs.groupby("etype").term.count(), dfc.groupby("etype").term.count()

In [ ]:
126/177

In [ ]:
# dfs[dfs["etype"] == -20]

In [ ]:
# dfc = df.copy()
# dfc.groupby("etype").head(2)

In [ ]:
# dfs = df.copy()
# dfs.groupby("etype").head(2)

In [ ]:
dfs[dfs["etype"] == -20].head()

In [ ]:
dfc[dfc["etype"] == -20].head()

In [ ]:
# Yeast acetylation, underrepresented, scipy
print(df.shape)
print(df.groupby("etype")["term"].count())

In [ ]:
# Yeast acetylation, underrepresented, cython
print(df.shape)
print(df.groupby("etype")["term"].count())

In [ ]:
# Yeast acetylation, both, scipy
print(df.shape)
print(df.groupby("etype")["term"].count())

In [ ]:
# Yeast acetylation, both, cython
print(df.shape)
print(df.groupby("etype")["term"].count())

In [ ]:
# Yeast acetylation, overrepresented, cython
print(df.shape)
print(df.groupby("etype")["term"].count())

In [ ]:
df[df["etype"] == -51]

In [ ]:
df.groupby("etype").head(2)

In [ ]:
# Problematic call
foreground_input = '511145.b1260%0d511145.b1261%0d511145.b1262%0d511145.b1263%0d511145.b1264%0d511145.b1812%0d511145.b2551%0d511145.b3117%0d511145.b3772%0d511145.b1015%0d511145.b2585'
foreground_input = foreground_input.split("%0d")
background_input = []

enrichment_method = "genome"
args_dict = {}
args_dict["enrichment_method"] = enrichment_method
args_dict["taxid"] = 511145
args_dict["FDR_cutoff"] = 0.05
args_dict["p_value_cutoff"] = 0.01
args_dict["limit_2_entity_type"] = None # "-20;-25;-26" #"-21;-22;-23;-51;-52;-53;-54;-55;-56-57;-58"
args_dict["filter_PMID_top_n"] = 100
args_dict["filter_foreground_count_one"] = False
args_dict["filter_parents"] = False
args_dict["go_slim_subset"] = None # "generic"
args_dict["o_or_u_or_both"] = "both" # "both" "underrepresented" "overrepresented"
args_dict["multiple_testing_per_etype"] = True
args_dict["privileged"] = True
args_dict["score_cutoff"] = 0
args_dict["caller_identity"] = "11_0"
taxid = args_dict["taxid"]
debug = False
profile = False
simplified_output = False
args_dict["simplified_output"] = simplified_output
args_dict["STRING_beta"] = True
KS_method = "cy"
# result = requests.post(url_, params={"output_format": "tsv", "enrichment_method": "genome", "taxid": 511145, "caller_identity": "11_0", "STRING_beta": True, 
#                                      'FDR_cutoff': '0.05'}, data={"foreground": fg, "background": bg})
ui = userinput.Userinput(pqo, fn=None, foreground_string=userinput.stringify_for_Userinput(foreground_input), background_string=userinput.stringify_for_Userinput(background_input), args_dict=args_dict)
preloaded_objects_per_analysis = pqo.get_preloaded_objects_per_analysis()
if debug:
    funcEnum_2_scores_dict_bg, foreground_n, background_n, fg_scores_matrix_data, fg_scores_matrix_indptr, bg_scores_matrix_data, bg_scores_matrix_indptr, p_values, cond_multitest, effectSizes, p_value_cutoff, funcEnum_count_foreground, funcEnum_count_background, over_under_int_arr, o_or_u_or_both_encoding, em, filter_foreground_count_one = run_enrichment_cy(ncbi, ui, preloaded_objects_per_analysis, static_preloaded_objects, low_memory=True, debug=debug, KS_method=KS_method, ENSP_2_tuple_funcEnum_score_dict=ENSP_2_tuple_funcEnum_score_dict)
else:
    df = run_enrichment_cy(ncbi, ui, preloaded_objects_per_analysis, static_preloaded_objects, low_memory=True, debug=debug, KS_method=KS_method, ENSP_2_tuple_funcEnum_score_dict=ENSP_2_tuple_funcEnum_score_dict)

In [ ]:
df.groupby("etype").head(2)

In [ ]:
# import requests
# from io import StringIO
# import pandas as pd
# # call api_help for help and argument defaults
# response = requests.get(r"https://agotool.org/api_help")
# print(response.json())
# url_ = r"https://agotool.org/api"
# # url_ = r"http://localhost:5000/api"

In [ ]:
# ENSPs = ['4932.YAR019C', '4932.YFR028C', '4932.YGR092W', '4932.YHR152W', '4932.YIL106W', '4932.YJL076W',
#      '4932.YLR079W', '4932.YML064C', '4932.YMR055C', '4932.YOR373W', '4932.YPR119W']
# fg = "%0d".join(ENSPs)
# result = requests.post(url_,
#                    params={"output_format": "tsv",
#                            "enrichment_method": "genome",
#                            "taxid": 559292, "STRING_beta": True}, 
#                        # UniProt reference proteomes uses "Saccharomyces cerevisiae S288C" with Taxid 559292 as a pan proteome instead of 4932 (TaxID on taxonomic rank of species).
#                    data={"foreground": fg})
# result = result.text
# df = pd.read_csv(StringIO(result), sep='\t')
# df.head(2)

In [ ]:
# import requests
# from io import StringIO
# import pandas as pd

# url_ = r"https://agotool.org/api"
# ENSPs = ['4932.YAR019C', '4932.YFR028C', '4932.YGR092W', '4932.YHR152W', '4932.YIL106W', '4932.YJL076W',
#      '4932.YLR079W', '4932.YML064C', '4932.YMR055C', '4932.YOR373W', '4932.YPR119W']
# fg = "%0d".join(ENSPs)
# result = requests.post(url_,
#                    params={"output_format": "tsv",
#                            "enrichment_method": "genome",
#                            "taxid": 559292, "STRING_beta": False}, 
#                        # UniProt reference proteomes uses "Saccharomyces cerevisiae S288C" with Taxid 559292 as a pan proteome instead of 4932 (TaxID on taxonomic rank of species).
#                    data={"foreground": fg})
# result = result.text
# df = pd.read_csv(StringIO(result), sep='\t')
# df.head(2)

In [ ]:
# ToDo PyTests 2 write
# compare ENSP_2_tuple_funcEnum_score_dict with Sparse Matrix (funcEnum_2_scores_dict_fg, funcEnum_2_scores_dict_bg). 
#   For any given input (of taxid 9606) check that FG_count <= BG_count
#   funcEnums of FG must also be in BG
#   scores of FG must also be in BG

# for given Protein what functions are associated?

In [ ]:
fg_scores_matrix, list_of_rowIndices_fg = slice_ScoresMatrix_for_given_ENSP(protein_ans_fg, ENSP_2_rowIndex_dict, CSC_ENSPencoding_2_FuncEnum)
fg_scores_matrix_data = fg_scores_matrix.data
fg_scores_matrix_indptr = fg_scores_matrix.indptr
set_fg_counts(fg_scores_matrix_data, fg_scores_matrix_indptr, funcEnum_count_foreground, filter_foreground_count_one)
# add_funcEnums_2_dict(protein_ans_fg, ENSP_2_functionEnumArray_dict, ENSP_2_tuple_funcEnum_score_dict)
add_funcEnums_2_dict_CSC(protein_ans_fg, ENSP_2_functionEnumArray_dict, ENSP_2_rowIndex_dict, CSR_ENSPencoding_2_FuncEnum)

In [ ]:
# How can the FG_count always be as high as FG_n?
# Why is BG_count not >= as FG_count? --> PyTest
funcEnum_2_scores_dict_bg = Taxid_2_FunctionEnum_2_Scores_dict[9606]

# The plan

In [ ]:
# e.g. GOCC:0043226; 1893 (funcEnum) with 59 BG_count, but 97 FG_count and enrichment_method "genome"
# --> PYTEST: FG_count should always be <= BG_count
# CHECK:
# Which files are in the pipeline? 
    # - Lars download with ENSPs
    # - translated to UniProtIDs, scaled values from float to int and backtracked
    # - counted per TaxID
    
# Which ENSPs are associated with GOCC:0043226 in original Lars download? 
# Which UniProtIDs are associated with GOCC:0043226 in original Lars download?
# How many and which

In [ ]:
fg_scores_matrix, list_of_rowIndices_fg = slice_ScoresMatrix_for_given_ENSP(protein_ans_fg, ENSP_2_rowIndex_dict, CSC_ENSPencoding_2_FuncEnum)
fg_scores_matrix_data = fg_scores_matrix.data
fg_scores_matrix_indptr = fg_scores_matrix.indptr

In [ ]:
ui.df_orig.shape

In [ ]:
collect_scores_per_term_limit_2_inclusionTerms(protein_ans_fg, ENSP_2_tuple_funcEnum_score_dict, funcEnums_2_include_set, list_2_array=True)

In [ ]:
prot_noFunc_list, scores_list = [], []
funcEnum = 1893
protein_ans_fg = ui.get_all_individual_foreground_ANs()
for prot in protein_ans_fg:
    try:
        funcEnum_arr, score_arr = ENSP_2_tuple_funcEnum_score_dict[prot]
    except:
        prot_noFunc_list.append(prot)
        continue
    x = np.where(funcEnum_arr == funcEnum)[0]
    if x.shape[0] == 0:
        print("")
        continue
    else:
        index_ = x[0]
    scores_list.append(score_arr[index_])

In [ ]:
len(scores_list), len(prot_noFunc_list)

In [ ]:
funcEnum_arr, score_arr = ENSP_2_tuple_funcEnum_score_dict["GPT_HUMAN"]
np.where(funcEnum_arr == funcEnum)
# index_ = np.where(funcEnum_arr == funcEnum)[0][0]
# print(score_arr[index_])

# Profiling

## cProfile

In [ ]:
del run_cythonized

In [ ]:
import run_cythonized
reload(run_cythonized)

In [ ]:
# import cProfile
# cProfile.run('run_cythonized.run_genome_cy(taxid, protein_ans, background_n, preloaded_objects_per_analysis, static_preloaded_objects, args_dict, low_memory=False)', sort='time') > prof_temp.txt

## line profiler

In [ ]:
## https://stackoverflow.com/questions/28301931/how-to-profile-cython-functions-line-by-line
## add this to Jupyter Notebook
#%%cython -f --compile-args=-DCYTHON_TRACE=1
#import Cython
######################################
### profiling # Set compiler directives (cf. http://docs.cython.org/src/reference/compilation.html)
import line_profiler
directive_defaults = Cython.Compiler.Options.get_directive_defaults() ### from Cython.Compiler.Options import directive_defaults # deprecated
directive_defaults['linetrace'] = True
directive_defaults['binding'] = True
######################################

## then run this to profile
# %load_ext line_profiler
import line_profiler

fn_userinput = r"/Users/dblyon/modules/cpr/agotool/data/exampledata/ExampleData_for_testing.txt"
foreground_input = ["HBA_HUMAN", "HBB_HUMAN", "HBD_HUMAN", "HBE_HUMAN"]
taxid = 9606
from_file = False

args_dict = {}
args_dict["enrichment_method"] = "genome"
args_dict["FDR_cutoff"] = 0.05
args_dict["p_value_cutoff"] = 0.01
args_dict["limit_2_entity_type"] = "-20;-25;-21" # "-20;-21;-22;-23;-25;-26" # None #"-21;-22;-23"
args_dict["filter_PMID_top_n"] = 50
args_dict["filter_foreground_count_one"] = True
args_dict["filter_parents"] = True
args_dict["go_slim_subset"] = None # "generic"
args_dict["taxid"] = taxid
args_dict["o_or_u_or_both"] = "both" # "both" "underrepresented"
args_dict["multiple_testing_per_etype"] = True
args_dict["privileged"] = True

background_input = query.get_proteins_of_taxid(taxid, read_from_flat_files=True)
if from_file:
    ui = userinput.Userinput(pqo, fn_userinput, args_dict=args_dict)
else:
    ui = userinput.Userinput(pqo, fn=None, foreground_string=userinput.stringify_for_Userinput(foreground_input), background_string=userinput.stringify_for_Userinput(background_input), args_dict=args_dict)
ncbi = pqo.ncbi

preloaded_objects_per_analysis = pqo.get_preloaded_objects_per_analysis()
profile = line_profiler.LineProfiler(run_enrichment_cy)
profile.runcall(run_enrichment_cy, ncbi, ui, preloaded_objects_per_analysis, static_preloaded_objects, low_memory=True)
profile.print_stats()
# > 70% of time spent on KS (Hemoglobin foreground, genome background). 76.7 (only categories with scores), 69.8 (all categories) 
#  --> test KS funtion alone

In [200]:
### testing calc_pvalues_new vs old
#%%cython -f --compile-args=-DCYTHON_TRACE=1
#import Cython
######################################
### profiling # Set compiler directives (cf. http://docs.cython.org/src/reference/compilation.html)
import line_profiler
directive_defaults = Cython.Compiler.Options.get_directive_defaults() ### from Cython.Compiler.Options import directive_defaults # deprecated
directive_defaults['linetrace'] = True
directive_defaults['binding'] = True
######################################

## then run this to profile
# %load_ext line_profiler
import line_profiler

fn_userinput = r"/Users/dblyon/modules/cpr/agotool/data/exampledata/ExampleData_for_testing.txt"
# foreground_input = ["HBA_HUMAN", "HBB_HUMAN", "HBD_HUMAN", "HBE_HUMAN"]
taxid = 9606
from_file = True
contiguous = True
foreground_n = 500
# foreground_input = sorted(get_random_human_ENSP(foreground_n, joined_for_web=False, contiguous=contiguous, UniProt_ID=True))

args_dict = {}
args_dict["enrichment_method"] = "genome"
args_dict["FDR_cutoff"] = 0.05
args_dict["p_value_cutoff"] = 0.01
args_dict["limit_2_entity_type"] = None # "-20;-25;-21" # "-20;-21;-22;-23;-25;-26" # None #"-21;-22;-23"
args_dict["filter_PMID_top_n"] = 50
args_dict["filter_foreground_count_one"] = True
args_dict["filter_parents"] = True
args_dict["go_slim_subset"] = None # "generic"
args_dict["taxid"] = taxid
args_dict["o_or_u_or_both"] = "both" # "both" "underrepresented"
args_dict["multiple_testing_per_etype"] = True
args_dict["privileged"] = True

background_input = query.get_proteins_of_taxid(taxid, read_from_flat_files=True)
if from_file:
    ui = userinput.Userinput(pqo, fn_userinput, args_dict=args_dict)
else:
    ui = userinput.Userinput(pqo, fn=None, foreground_string=userinput.stringify_for_Userinput(foreground_input), background_string=userinput.stringify_for_Userinput(background_input), args_dict=args_dict)
ncbi = pqo.ncbi

preloaded_objects_per_analysis = pqo.get_preloaded_objects_per_analysis()
profile = line_profiler.LineProfiler(run_enrichment_cy)
profile.runcall(run_enrichment_cy, ncbi, ui, preloaded_objects_per_analysis, static_preloaded_objects, low_memory=True, pvalue_version="new")
profile.print_stats()
# > 70% of time spent on KS (Hemoglobin foreground, genome background). 76.7 (only categories with scores), 69.8 (all categories) 
#  --> test KS funtion alone

running calc_pvalues_v4
Timer unit: 1e-06 s

Total time: 2.12591 s
File: /Users/dblyon/.ipython/cython/_cython_magic_2fd9ca079481bd97d0ed7422c29f4762.pyx
Function: run_enrichment_cy at line 106

Line #      Hits         Time  Per Hit   % Time  Line Contents
   106                                           def run_enrichment_cy(ncbi, ui, preloaded_objects_per_analysis, static_preloaded_objects, low_memory=False, debug=False, pvalue_version="old"):
   107         1          2.0      2.0      0.0      if not low_memory:
   108                                                   ENSP_2_functionEnumArray_dict, year_arr, hierlevel_arr, entitytype_arr, functionalterm_arr, indices_arr, description_arr, category_arr, etype_2_minmax_funcEnum, function_enumeration_len, etype_cond_dict, etype_2_num_functions_dict, taxid_2_proteome_count, taxid_2_tuple_funcEnum_index_2_associations_counts, lineage_dict_enum, blacklisted_terms_bool_arr, cond_etypes_with_ontology, cond_etypes_rem_foreground_ids, kegg_t

In [ ]:
### get objects for code block below
foreground_input = ["HBA_HUMAN", "HBB_HUMAN", "HBD_HUMAN", "HBE_HUMAN"] # example 1: Human hemoglobin, genome
enrichment_method = "genome" # "characterize_foreground" "abundance_correction" "compare_samples" "genome" "compare_groups"
from_file = False
args_dict = {}
args_dict["enrichment_method"] = enrichment_method
args_dict["taxid"] = 9606
args_dict["FDR_cutoff"] = 0.05
args_dict["p_value_cutoff"] = 0.01
args_dict["limit_2_entity_type"] = "-20;-25;-26" #"-20;-25;-21" # "-20;-21;-22;-23;-25;-26" # None #"-21;-22;-23"
args_dict["filter_PMID_top_n"] = 50
args_dict["filter_foreground_count_one"] = True
args_dict["filter_parents"] = True
args_dict["go_slim_subset"] = None # "generic"
args_dict["o_or_u_or_both"] = "overrepresented" # "both" "underrepresented"
args_dict["multiple_testing_per_etype"] = True
args_dict["privileged"] = True
args_dict["score_cutoff"] = 0
args_dict["foreground_replicates"] = 10
args_dict["background_replicates"] = 10
taxid = args_dict["taxid"]
background_input = query.get_proteins_of_taxid(taxid, read_from_flat_files=True)
if from_file:
    ui = userinput.Userinput(pqo, fn_userinput, args_dict=args_dict)
else:
    ui = userinput.Userinput(pqo, fn=None, foreground_string=userinput.stringify_for_Userinput(foreground_input), background_string=userinput.stringify_for_Userinput(background_input), args_dict=args_dict)
ncbi = pqo.ncbi
preloaded_objects_per_analysis = pqo.get_preloaded_objects_per_analysis()
foreground_n, background_n, funcEnum_2_scores_dict_fg, funcEnum_2_scores_dict_bg, p_values, cond_multitest, effectSizes, p_value_cutoff, funcEnum_count_foreground, funcEnum_count_background, over_under_int_arr, o_or_u_or_both_encoding = run_enrichment_cy(ncbi, ui, preloaded_objects_per_analysis, static_preloaded_objects, low_memory=True)

In [ ]:
from scipy.stats import distributions
def ks_2samp_dbl(data1, data2):
    data1 = np.sort(data1)
    data2 = np.sort(data2)
    data1 = np.array(data1)
    data2 = np.array(data2)
    n1 = data1.shape[0]
    n2 = data2.shape[0]
    data_all = np.concatenate([data1, data2])
    cdf1 = np.searchsorted(data1, data_all, side='right') / n1
    cdf2 = np.searchsorted(data2, data_all, side='right') / n2
    d = np.max(np.absolute(cdf1 - cdf2))
    # Note: d absolute not signed distance
    en = np.sqrt(n1 * n2 / (n1 + n2))
    try:
        prob = distributions.kstwobign.sf((en + 0.12 + 0.11 / en) * d)
    # except Exception:
        # warnings.warn('This should not happen! Please open an issue at '
        #             'https://github.com/scipy/scipy/issues and provide the code '
        #            'you used to trigger this warning.\n')
        # prob = 1.0
    except:
        print("This shouldn't happen")
        raise StopIteration

    # return Ks_2sampResult(d, prob)
    return d, prob

In [ ]:
### fill with zeros for proper calculation and performance evaluation
funcEnum = 68613
scores_fg = funcEnum_2_scores_dict_fg[funcEnum]
scores_bg = funcEnum_2_scores_dict_bg[funcEnum]
scores_bg = list(scores_bg) # for genome method --> not the place to do this        
len_scores_fg = len(scores_fg)
number_of_zeros_2_fill = foreground_n - len_scores_fg
if number_of_zeros_2_fill > 0:
    scores_fg = [0]*number_of_zeros_2_fill + scores_fg
len_scores_bg = len(scores_bg)
number_of_zeros_2_fill = background_n - len_scores_bg
if number_of_zeros_2_fill > 0:
    scores_bg = [0]*number_of_zeros_2_fill + scores_bg    
data1, data2 = scores_fg, scores_bg

d_dbl, prob_dbl = ks_2samp_dbl(data1, data2)
d, prob = stats.ks_2samp(data1, data2, alternative="two-sided", mode="asymp")
print(d_dbl, prob_dbl)
assert d_dbl == d
# assert prob_dbl == prob
prob_dbl == prob

In [ ]:
def list_of_floats_2_R_style(vals, max_num_vals_per_line=100):
    vals = [str(ele) for ele in vals]
    string_2_return = "<-c("
    while len(vals) > 0:
        string_2_return += "{},\n".format(", ".join(vals[:max_num_vals_per_line]))
        vals = vals[max_num_vals_per_line:]
    return string_2_return[:-2] + ")"

In [ ]:
def transformation_to_randomize_sort_order(fg, bg):
    # value = value * ((double) 1.0 + ((double) rand()) / (((double) RAND_MAX) * (double) 1000.0)); // transformation of fold_change values in order to randomize sort order in case of ties
    np.random.seed(12345)
#     fg = [value * (1.0 + np.random.randint(0, 2147483647) / (2147483647.0 * 1000.0)) for value in fg]
    # bg = [value * (1.0 + np.random.randint(0, 2147483647) / (2147483647.0 * 1000.0)) for value in bg]
    fg = [value + np.random.randint(0, 2147483647) / (2147483647.0 * 1000.0) for value in fg]
    bg = [value + np.random.randint(0, 2147483647) / (2147483647.0 * 1000.0) for value in bg]
    return fg, bg

In [ ]:
funcEnum = 68613
scores_fg = funcEnum_2_scores_dict_fg[funcEnum]
scores_bg = funcEnum_2_scores_dict_bg[funcEnum]
print(stats.ks_2samp(scores_fg, scores_bg, alternative="two-sided", mode="exact"))
scores_fg, scores_bg = transformation_to_randomize_sort_order(scores_fg, scores_bg)
print(stats.ks_2samp(scores_fg, scores_bg, alternative="two-sided", mode="exact"))
print(stats.ks_2samp(data1, data2, alternative="two-sided", mode="exact"))
data1, data2 = transformation_to_randomize_sort_order(data1, data2)
print(stats.ks_2samp(data1, data2, alternative="two-sided", mode="exact"))

In [ ]:
%matplotlib inline
dfx = pd.DataFrame()
dfx["bg"] = pd.Series(scores_bg)
dfx["bg"].hist(cumulative=True, density=1, bins=100, alpha=0.2)
dfx["fg"] = pd.Series(scores_fg)
dfx["fg"].hist(cumulative=True, density=1, bins=100, alpha=0.2)

In [ ]:
def fg_and_bg_2_DF_melt_and_add_ranks(fg, bg):
    dfx = pd.DataFrame()
    dfx["bg"] = pd.Series(bg)
    dfx["fg"] = pd.Series(fg)
    # transform dataframe to long format
    dfxm = dfx.melt(value_vars=["fg", "bg"], var_name="FG_BG", value_name="score")
    dfxm = dfxm[dfxm["score"].notnull()]
    dfxm = dfxm.sort_values(["score", "FG_BG"], ascending=[True, False]).reset_index(drop=True)
    dfxm["rank"] = dfxm["score"].rank(method="first")
    return dfxm
# scores_fg = funcEnum_2_scores_dict_fg[funcEnum]
# scores_bg = funcEnum_2_scores_dict_bg[funcEnum]
# dfxm = fg_and_bg_2_DF_melt_and_add_ranks(scores_fg, scores_bg)
# print(stats.ks_2samp(scores_fg, scores_bg, alternative="two-sided", mode="exact"))
# stats.ks_2samp(dfxm.loc[dfxm["FG_BG"] == "fg", "score"].to_list(), dfxm.loc[dfxm["FG_BG"] == "bg", "score"].to_list(), alternative="two-sided", mode="exact")

In [ ]:
# scores_fg = funcEnum_2_scores_dict_fg[funcEnum]
# sorted(scores_fg)

In [ ]:
### Print profiling statistics using the `line_profiler` API
### 'run_cythonized.run_genome_cy(taxid, protein_ans, background_n, preloaded_objects_per_analysis, static_preloaded_objects, args_dict, low_memory=False)'
profile = line_profiler.LineProfiler(run_genome_cy)
profile.runcall(run_genome_cy, taxid, protein_ans, background_n, preloaded_objects_per_analysis, static_preloaded_objects, args_dict, low_memory=False)
profile.print_stats()